<a href="https://colab.research.google.com/github/tsm-mehmetakiftasoz/tsm_makif/blob/main/%D0%93%D0%BE%D1%82%D0%BE%D0%B2%D0%BE_%D0%92%D1%81%D0%B5%20%D0%B2%20%D0%BE%D0%B4%D0%BD%D0%BE%D0%BC_%20beta_test_ver3-3%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Предназначено для объединения всех отчетов. Имена должны быть строго идентичными, и при выгрузке из SAP они должны быть в формате MAKIF_REPORT, иначе код не будет работать!!!

Необходимые отчеты:
*   ME5A_2022, ME5A_2023, ME5A_2024, ME5A_2025,ME5A_2026 (могут быть выгружены из SAP в формате 01.01.202X-31.12.202X)
*   ME2N_2022, ME2N_2023, ME2N_2024, ME2N_2025,ME2N_2026 (могут быть выгружены из SAP в формате 01.01.202X-31.12.202X)
*   zmm059_2022, zmm059_2023, zmm059_2025,zmm059_2026, zmm059_2024_1, zmm059_2024_2, zmm059_2024_3, zmm059_2024_4.(Отчеты за 2022, 2023, 2025 годы могут быть выгружены из SAP в формате 01.01.202X-31.12.202X. Для 2024 года граничные даты: 23.04.2024, 15.07.2024, 29.11.2024 и 01.12.2024 по 31.12.2024 (на 30.11 система выдает ошибку)).
*   zmb51_2022, zmb51_2023, zmb51_2025_1 (01.01.25-31.05.25), zmb51_2025_2 (01.06.2025-31.12.2025),zmb51_2026, zmb51_2024_1 (01.01-30.04.24), zmb51_2024_2 (01.05.2024-31.07.2024), zmb51_2024_3 (01.08.2024-30.09.2024), zmb51_2024_4 (01.10.2024-30.11.2024), zmb51_2024_5 (01.12.2024-31.12.2024)
*   ZMM067_2022(01.01.202X-31.12.202X) , ZMM067_2023(01.01.202X-31.12.202X), ZMM067_2024_1(01.01-31.05.24),ZMM067_2024_2(01.06-31.12.24),ZMM067_2025(01.01.202X-31.12.202X),ZMM067_2026

In [1]:
!pip install pandas
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 4.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from google.colab import files
import os
import zipfile

In [3]:
def duzenle_sayi_sutunu(df, sutun_adi):
    """
    Sayısal değerleri metin olarak tutan ve binlik/ondalık ayracı içeren sütunu dönüştürür.
    - Nokta (.) binlik ayracı olarak kaldırılır
    - Virgül (,) ondalık ayracı olarak noktaya çevrilir
    - Float olarak dönüştürülür
    - Eğer tam sayı ise integer olarak, değilse float olarak kalır
    - Sonuç string olarak döndürülür (görsel temizlik için)

    Parametreler:
    df : pd.DataFrame
    sutun_adi : str

    Dönüş:
    df (işlenmiş hali)
    """
    df[sutun_adi] = (
        df[sutun_adi]
        .astype(str)
        .str.replace(" ", "", regex=False)
        .str.replace(".", "", regex=False)
        .str.replace(",", ".", regex=False)
        .astype(float)
        .apply(lambda x: str(int(x)) if x.is_integer() else str(x))
    )
    return df


In [4]:
def temizle_ve_cevir(x):
    try:
        return str(int(float(str(x).strip())))
    except:
        return str(x).strip()


In [5]:
# Temizlenecek değerler listesi
temizlenecekler = ["nan", "Nan", "NaN", ""," ","0","naN","nAn","NAn","nAN"]

def temizle_nan(value):
    if str(value).strip().lower() in [x.lower() for x in temizlenecekler]:
        return pd.NA
    return value


In [6]:
# Güvenli filtreleme fonksiyonu
def dolu_mi(x):
    return pd.notna(x) and str(x).strip() != "" and str(x).strip().upper() != "<NA>"


In [7]:
#me5a'nın yüklenmesi
df_me5a_2022 = pd.read_csv('/content/ME5A_2022.csv', sep=None, engine='python')
df_me5a_2023 = pd.read_csv('/content/ME5A_2023.csv', sep=None, engine='python')
df_me5a_2024 = pd.read_csv('/content/ME5A_2024.csv', sep=None, engine='python')
df_me5a_2025 = pd.read_csv('/content/ME5A_2025.csv', sep=None, engine='python')
df_me5a_2026 = pd.read_csv('/content/ME5A_2026.csv', sep=None, engine='python')
# sep=None will automatically detect the delimiter
# engine='python' is slower but more robust for complex cases
#zmm059un yüklenmesi
df_zmm059_2022 = pd.read_csv('/content/zmm059_2022.csv', sep=None, engine='python')
df_zmm059_2023 = pd.read_csv('/content/zmm059_2023.csv', sep=None, engine='python')
df_zmm059_2024_1 = pd.read_csv('/content/zmm059_2024_1.csv', sep=None, engine='python')
df_zmm059_2024_2 = pd.read_csv('/content/zmm059_2024_2.csv', sep=None, engine='python')
df_zmm059_2024_3 = pd.read_csv('/content/zmm059_2024_3.csv', sep=None, engine='python')
df_zmm059_2024_4 = pd.read_csv('/content/zmm059_2024_4.csv', sep=None, engine='python')
df_zmm059_2025 = pd.read_csv('/content/zmm059_2025.csv', sep=None, engine='python')
df_zmm059_2026 = pd.read_csv('/content/zmm059_2026.csv', sep=None, engine='python')
#yardımcı tabloların yüklenmesi
df_tedarikci_listesi = pd.read_csv('/content/tedarikci_isim.csv', sep=None, engine='python')
df_material_list = pd.read_csv('/content/material_list.csv', sep=None, engine='python')
# sep=None will automatically detect the delimiter
# engine='python' is slower but more robust for complex cases
#me2n'in yüklenmesi
df_me2n_2022 = pd.read_csv('/content/ME2N_2022.csv', sep=None, engine='python')
df_me2n_2023 = pd.read_csv('/content/ME2N_2023.csv', sep=None, engine='python')
df_me2n_2024 = pd.read_csv('/content/ME2N_2024.csv', sep=None, engine='python')
df_me2n_2025 = pd.read_csv('/content/ME2N_2025.csv', sep=None, engine='python')
df_me2n_2026 = pd.read_csv('/content/ME2N_2026.csv', sep=None, engine='python')
# sep=None will automatically detect the delimiter
# engine='python' is slower but more robust for complex cases
#zmb51'in yüklenmesi
df_zmb51_2022 = pd.read_csv('/content/zmb51_2022.csv', sep=None, engine='python')
df_zmb51_2023 = pd.read_csv('/content/zmb51_2023.csv', sep=None, engine='python')
df_zmb51_2024_1 = pd.read_csv('/content/zmb51_2024_1.csv', sep=None, engine='python')
df_zmb51_2024_2 = pd.read_csv('/content/zmb51_2024_2.csv', sep=None, engine='python')
df_zmb51_2024_3 = pd.read_csv('/content/zmb51_2024_3.csv', sep=None, engine='python')
df_zmb51_2024_4 = pd.read_csv('/content/zmb51_2024_4.csv', sep=None, engine='python')
df_zmb51_2024_5 = pd.read_csv('/content/zmb51_2024_5.csv', sep=None, engine='python')
#df_zmb51_2024_6 = pd.read_csv('/content/zmb51_2024_6.csv', sep=None, engine='python')
#df_zmb51_2024_7 = pd.read_csv('/content/zmb51_2024_7.csv', sep=None, engine='python')
df_zmb51_2025 = pd.read_csv('/content/zmb51_2025.csv', sep=None, engine='python')
df_zmb51_2025_2 = pd.read_csv('/content/zmb51_2025-2.csv', sep=None, engine='python')
df_zmb51_2026 = pd.read_csv('/content/zmb51_2026.csv', sep=None, engine='python')
# sep=None will automatically detect the delimiter
# engine='python' is slower but more robust for complex cases
#zmm067'in yüklenmesi
df_zmm067_2022 = pd.read_csv('/content/ZMM067_2022.csv', sep=None, engine='python')
df_zmm067_2023 = pd.read_csv('/content/ZMM067_2023.csv', sep=None, engine='python')
df_zmm067_2024_1 = pd.read_csv('/content/ZMM067_2024_1.csv', sep=None, engine='python')
df_zmm067_2024_2 = pd.read_csv('/content/ZMM067_2024_2.csv', sep=None, engine='python')
df_zmm067_2025 = pd.read_csv('/content/ZMM067_2025.csv', sep=None, engine='python')
df_zmm067_2026 = pd.read_csv('/content/ZMM067_2026.csv', sep=None, engine='python')
#
#zpp001 yüklenmesi
zpp001 = pd.read_csv('/content/ZPP001.csv', sep=None, engine='python')



In [8]:
#ilk sutuna yıl yazdırma me5a
df_me5a_2022["ГОД"] = "2022"
cols = ["ГОД"] + [col for col in df_me5a_2022.columns if col != "ГОД"]
df_me5a_2022 = df_me5a_2022[cols]
#
df_me5a_2023["ГОД"] = "2023"
cols = ["ГОД"] + [col for col in df_me5a_2023.columns if col != "ГОД"]
df_me5a_2023 = df_me5a_2023[cols]
#
df_me5a_2024["ГОД"] = "2024"
cols = ["ГОД"] + [col for col in df_me5a_2024.columns if col != "ГОД"]
df_me5a_2024 = df_me5a_2024[cols]
#
df_me5a_2025["ГОД"] = "2025"
cols = ["ГОД"] + [col for col in df_me5a_2025.columns if col != "ГОД"]
df_me5a_2025 = df_me5a_2025[cols]
#
df_me5a_2026["ГОД"] = "2026"
cols = ["ГОД"] + [col for col in df_me5a_2026.columns if col != "ГОД"]
df_me5a_2026 = df_me5a_2026[cols]

#ilk sutuna yıl yazdırma zmm059
df_zmm059_2022["ГОД"] = "2022"
cols = ["ГОД"] + [col for col in df_zmm059_2022.columns if col != "ГОД"]
df_zmm059_2022 = df_zmm059_2022[cols]
#
df_zmm059_2023["ГОД"] = "2023"
cols = ["ГОД"] + [col for col in df_zmm059_2023.columns if col != "ГОД"]
df_zmm059_2023 = df_zmm059_2023[cols]
#
df_zmm059_2025["ГОД"] = "2025"
cols = ["ГОД"] + [col for col in df_zmm059_2025.columns if col != "ГОД"]
df_zmm059_2025 = df_zmm059_2025[cols]
#
df_zmm059_2024_1["ГОД"] = "2024"
cols = ["ГОД"] + [col for col in df_zmm059_2024_1.columns if col != "ГОД"]
df_zmm059_2024_1 = df_zmm059_2024_1[cols]
#
df_zmm059_2024_2["ГОД"] = "2024"
cols = ["ГОД"] + [col for col in df_zmm059_2024_2.columns if col != "ГОД"]
df_zmm059_2024_2 = df_zmm059_2024_2[cols]
#
df_zmm059_2024_3["ГОД"] = "2024"
cols = ["ГОД"] + [col for col in df_zmm059_2024_3.columns if col != "ГОД"]
df_zmm059_2024_3 = df_zmm059_2024_3[cols]
#
df_zmm059_2024_4["ГОД"] = "2024"
cols = ["ГОД"] + [col for col in df_zmm059_2024_4.columns if col != "ГОД"]
df_zmm059_2024_4 = df_zmm059_2024_4[cols]
#
df_zmm059_2026["ГОД"] = "2026"
cols = ["ГОД"] + [col for col in df_zmm059_2026.columns if col != "ГОД"]
df_zmm059_2026 = df_zmm059_2026[cols]

#ilk sutuna yıl yazdırma me2n
df_me2n_2022["ГОД"] = "2022"
cols = ["ГОД"] + [col for col in df_me2n_2022.columns if col != "ГОД"]
df_me2n_2022 = df_me2n_2022[cols]
#
df_me2n_2023["ГОД"] = "2023"
cols = ["ГОД"] + [col for col in df_me2n_2023.columns if col != "ГОД"]
df_me2n_2023 = df_me2n_2023[cols]
#
df_me2n_2024["ГОД"] = "2024"
cols = ["ГОД"] + [col for col in df_me2n_2024.columns if col != "ГОД"]
df_me2n_2024 = df_me2n_2024[cols]
#
df_me2n_2025["ГОД"] = "2025"
cols = ["ГОД"] + [col for col in df_me2n_2025.columns if col != "ГОД"]
df_me2n_2025 = df_me2n_2025[cols]
#
df_me2n_2026["ГОД"] = "2026"
cols = ["ГОД"] + [col for col in df_me2n_2026.columns if col != "ГОД"]
df_me2n_2026 = df_me2n_2026[cols]
#ilk sutuna yıl yazdırma zmb51
df_zmb51_2022["ГОД"] = "2022"
cols = ["ГОД"] + [col for col in df_zmb51_2022.columns if col != "ГОД"]
df_zmb51_2022 = df_zmb51_2022[cols]
#
df_zmb51_2023["ГОД"] = "2023"
cols = ["ГОД"] + [col for col in df_zmb51_2023.columns if col != "ГОД"]
df_zmb51_2023 = df_zmb51_2023[cols]
#
df_zmb51_2024_1["ГОД"] = "2024"
cols = ["ГОД"] + [col for col in df_zmb51_2024_1.columns if col != "ГОД"]
df_zmb51_2024_1 = df_zmb51_2024_1[cols]
#
df_zmb51_2024_2["ГОД"] = "2024"
cols = ["ГОД"] + [col for col in df_zmb51_2024_2.columns if col != "ГОД"]
df_zmb51_2024_2 = df_zmb51_2024_2[cols]
#
df_zmb51_2024_3["ГОД"] = "2024"
cols = ["ГОД"] + [col for col in df_zmb51_2024_3.columns if col != "ГОД"]
df_zmb51_2024_3 = df_zmb51_2024_3[cols]
#
df_zmb51_2024_4["ГОД"] = "2024"
cols = ["ГОД"] + [col for col in df_zmb51_2024_4.columns if col != "ГОД"]
df_zmb51_2024_4 = df_zmb51_2024_4[cols]
#
df_zmb51_2024_5["ГОД"] = "2024"
cols = ["ГОД"] + [col for col in df_zmb51_2024_5.columns if col != "ГОД"]
df_zmb51_2024_5 = df_zmb51_2024_5[cols]
#
df_zmb51_2025["ГОД"] = "2025"
cols = ["ГОД"] + [col for col in df_zmb51_2025.columns if col != "ГОД"]
df_zmb51_2025 = df_zmb51_2025[cols]
#
df_zmb51_2025_2["ГОД"] = "2025"
cols = ["ГОД"] + [col for col in df_zmb51_2025_2.columns if col != "ГОД"]
df_zmb51_2025_2 = df_zmb51_2025_2[cols]
#
df_zmb51_2026["ГОД"] = "2026"
cols = ["ГОД"] + [col for col in df_zmb51_2026.columns if col != "ГОД"]
df_zmb51_2026 = df_zmb51_2026[cols]
#ilk sutuna yıl yazdırma zmm067
# 2022
df_zmm067_2022["ГОД"] = "2022"
cols = ["ГОД"] + [col for col in df_zmm067_2022.columns if col != "ГОД"]
df_zmm067_2022 = df_zmm067_2022[cols]

# 2023
df_zmm067_2023["ГОД"] = "2023"
cols = ["ГОД"] + [col for col in df_zmm067_2023.columns if col != "ГОД"]
df_zmm067_2023 = df_zmm067_2023[cols]

# 2024_1
df_zmm067_2024_1["ГОД"] = "2024"
cols = ["ГОД"] + [col for col in df_zmm067_2024_1.columns if col != "ГОД"]
df_zmm067_2024_1 = df_zmm067_2024_1[cols]

# 2024_2
df_zmm067_2024_2["ГОД"] = "2024"
cols = ["ГОД"] + [col for col in df_zmm067_2024_2.columns if col != "ГОД"]
df_zmm067_2024_2 = df_zmm067_2024_2[cols]

# 2025
df_zmm067_2025["ГОД"] = "2025"
cols = ["ГОД"] + [col for col in df_zmm067_2025.columns if col != "ГОД"]
df_zmm067_2025 = df_zmm067_2025[cols]
# 2026
df_zmm067_2026["ГОД"] = "2026"
cols = ["ГОД"] + [col for col in df_zmm067_2026.columns if col != "ГОД"]
df_zmm067_2026 = df_zmm067_2026[cols]

In [9]:
#me5a birleştirme
df_listesi_me5a = [df_me5a_2026,df_me5a_2025,df_me5a_2024, df_me5a_2023,df_me5a_2022 ]
df_me5a_2022_2025 = pd.concat(df_listesi_me5a, ignore_index=True)
# zmm059 birleştirme
df_listesi_zmm059 = [df_zmm059_2026,df_zmm059_2025, df_zmm059_2024_4, df_zmm059_2024_3, df_zmm059_2024_2, df_zmm059_2024_1, df_zmm059_2023,df_zmm059_2022 ]
df_zmm059_2022_2025 = pd.concat(df_listesi_zmm059, ignore_index=True)
#me2n birleştirme
df_listesi_me2n = [df_me2n_2026,df_me2n_2025, df_me2n_2024, df_me2n_2023,df_me2n_2022]
df_me2n_2022_2025 = pd.concat(df_listesi_me2n, ignore_index=True)
#zmb51 birleştirme
df_listesi_zmb51 = [df_zmb51_2026,df_zmb51_2025_2,df_zmb51_2025,df_zmb51_2024_5,df_zmb51_2024_4,df_zmb51_2024_3,
                    df_zmb51_2024_2,df_zmb51_2024_1,df_zmb51_2023,df_zmb51_2022
    ]
birlesik_df_zmb51 = pd.concat(df_listesi_zmb51, ignore_index=True)
#zmm067 birleştirme
df_listesi_zmm067 = [df_zmm067_2026,df_zmm067_2025,df_zmm067_2024_2,df_zmm067_2024_1,df_zmm067_2023,df_zmm067_2022]
df_zmm067_2022_2025 = pd.concat(df_listesi_zmm067, ignore_index=True)

In [10]:
#yedek tablolar
df_me5a_2022_2025_yedek=df_me5a_2022_2025.copy()
df_zmm059_2022_2025_yedek=df_zmm059_2022_2025.copy()
df_me2n_2022_2025_yedek=df_me2n_2022_2025.copy()
birlesik_df_zmb51_yedek=birlesik_df_zmb51.copy()
df_zmm067_2022_2025_yedek=df_zmm067_2022_2025.copy()

In [11]:
#zmm059
df_zmm059_2022_2025["﻿Документ закупки"] = df_zmm059_2022_2025["﻿Документ закупки"].apply(temizle_ve_cevir)
df_zmm059_2022_2025.rename(columns={'﻿Документ закупки': 'Документ_закупки'}, inplace=True)
df_zmm059_2022_2025["Материал"] = df_zmm059_2022_2025["Материал"].apply(temizle_ve_cevir)
df_zmm059_2022_2025["Заявка"] = df_zmm059_2022_2025["Заявка"].apply(temizle_ve_cevir)
df_zmm059_2022_2025 = duzenle_sayi_sutunu(df_zmm059_2022_2025, "Количество ЗП")
df_zmm059_2022_2025 = duzenle_sayi_sutunu(df_zmm059_2022_2025, "Минимальная цена ЗП")
df_zmm059_2022_2025 = duzenle_sayi_sutunu(df_zmm059_2022_2025, "Единица ЗП")
df_zmm059_2022_2025 = duzenle_sayi_sutunu(df_zmm059_2022_2025, "Цена нетто ЗП")
df_zmm059_2022_2025 = duzenle_sayi_sutunu(df_zmm059_2022_2025, "Итого цена нетто")
df_zmm059_2022_2025["Дата создания"] = pd.to_datetime(df_zmm059_2022_2025["Дата создания"], format="%d.%m.%Y", errors="coerce").dt.date
df_zmm059_2022_2025["Дата поставки"] = pd.to_datetime(df_zmm059_2022_2025["Дата поставки"], format="%d.%m.%Y", errors="coerce").dt.date
df_zmm059_2022_2025["Дата доставки, одобренная Поставщиком"] = pd.to_datetime(df_zmm059_2022_2025["Дата доставки, одобренная Поставщиком"], format="%d.%m.%Y", errors="coerce").dt.date
df_zmm059_2022_2025["Группа закупок"] = df_zmm059_2022_2025["Группа закупок"].apply(temizle_ve_cevir)
df_zmm059_2022_2025["БЕ"] = df_zmm059_2022_2025["БЕ"].apply(temizle_ve_cevir)
df_zmm059_2022_2025["Закуп. организация"] = df_zmm059_2022_2025["Закуп. организация"].apply(temizle_ve_cevir)
df_zmm059_2022_2025["Поставщик"] = df_zmm059_2022_2025["Поставщик"].apply(temizle_ve_cevir)
df_zmm059_2022_2025["Дата создания.1"] = pd.to_datetime(df_zmm059_2022_2025["Дата создания.1"], format="%d.%m.%Y", errors="coerce").dt.date
df_zmm059_2022_2025 = duzenle_sayi_sutunu(df_zmm059_2022_2025, "Цена заказа нетто")
df_zmm059_2022_2025 = duzenle_sayi_sutunu(df_zmm059_2022_2025, "Стоимость брутто")
df_zmm059_2022_2025 = duzenle_sayi_sutunu(df_zmm059_2022_2025, "Недопоставленное количество")
df_zmm059_2022_2025 = duzenle_sayi_sutunu(df_zmm059_2022_2025, "Количество поставки")
df_zmm059_2022_2025 = duzenle_sayi_sutunu(df_zmm059_2022_2025, "Сумма брутто")
df_zmm059_2022_2025 = duzenle_sayi_sutunu(df_zmm059_2022_2025, "Количество по счету")
df_zmm059_2022_2025["Дата проводки"] = pd.to_datetime(df_zmm059_2022_2025["Дата проводки"], format="%d.%m.%Y", errors="coerce").dt.date
df_zmm059_2022_2025 = duzenle_sayi_sutunu(df_zmm059_2022_2025, "Количество по счету.1")
df_zmm059_2022_2025 = duzenle_sayi_sutunu(df_zmm059_2022_2025, "Остаток количества по счету")
df_zmm059_2022_2025["Дата контракта"] = pd.to_datetime(df_zmm059_2022_2025["Дата контракта"], format="%d.%m.%Y", errors="coerce").dt.date
df_zmm059_2022_2025["Дата вступления в силу контракта"] = pd.to_datetime(df_zmm059_2022_2025["Дата вступления в силу контракта"], format="%d.%m.%Y", errors="coerce").dt.date
df_zmm059_2022_2025["Дата окончания действия договора"] = pd.to_datetime(df_zmm059_2022_2025["Дата окончания действия договора"], format="%d.%m.%Y", errors="coerce").dt.date


In [12]:
df_me5a_2022_2025.columns

Index(['ГОД', '﻿Заявка', 'Позиция заявки', 'Материал', 'Индикатор удаления',
       'Затребовал', 'Индик. выдачи', 'Группа деблокир.',
       'Статус обработки заявки', 'Дата заявки', 'Группа закупок',
       'Дата изменения', 'Заказ на поставку', 'Позиция ЗкзНаПостав',
       'Дата заказа', 'Долгосрочный договор', 'ЗатребованКолич',
       'Заказанное к-во', 'Единица измерения', 'RU Наименование',
       'TR Наименование', 'Инвентарный номер', 'Краткий текст',
       'Закуп. организация', 'Создал', 'ПоступлМатериала', 'Поступление счета',
       'Номер проекта', 'Группа материалов', 'Дата поставки',
       'Завод-поставщик', 'Вид документа', 'Статус обработки',
       'Индикатор создания', 'Стратегия деблокир.', 'НДП-материал',
       'Номер потребности', 'Объем дефицита', 'Тип даты поставки',
       'Дата деблокирования'],
      dtype='object')

In [13]:
#ME5A
df_me5a_2022_2025["﻿Заявка"] = df_me5a_2022_2025["﻿Заявка"].apply(temizle_ve_cevir)
df_me5a_2022_2025.rename(columns={'﻿Заявка': 'Заявка'}, inplace=True)
df_me5a_2022_2025.rename(columns={'Индикатор удаления': 'Индикатор удаления_B3'}, inplace=True)
df_me5a_2022_2025["Материал"] = df_me5a_2022_2025["Материал"].apply(temizle_ve_cevir)
df_me5a_2022_2025["'Дата заявки"] = pd.to_datetime(df_me5a_2022_2025["Дата заявки"], format="%d.%m.%Y", errors="coerce").dt.date
df_me5a_2022_2025["Группа закупок"] = df_me5a_2022_2025["Группа закупок"].apply(temizle_ve_cevir)
df_me5a_2022_2025["'Дата заявки"] = pd.to_datetime(df_me5a_2022_2025["Дата заявки"], format="%d.%m.%Y", errors="coerce").dt.date
df_me5a_2022_2025["Дата изменения"] = df_me5a_2022_2025["Дата изменения"].apply(temizle_ve_cevir)
df_me5a_2022_2025["Заказ на поставку"] = df_me5a_2022_2025["Заказ на поставку"].apply(temizle_ve_cevir)
df_me5a_2022_2025["Позиция ЗкзНаПостав"] = df_me5a_2022_2025["Позиция ЗкзНаПостав"].apply(temizle_ve_cevir)
df_me5a_2022_2025 = duzenle_sayi_sutunu(df_me5a_2022_2025, "ЗатребованКолич")
df_me5a_2022_2025 = duzenle_sayi_sutunu(df_me5a_2022_2025, "Заказанное к-во")
df_me5a_2022_2025["Закуп. организация"] = df_me5a_2022_2025["Закуп. организация"].apply(temizle_ve_cevir)
df_me5a_2022_2025["Номер проекта"] = df_me5a_2022_2025["Номер проекта"].apply(temizle_ve_cevir)
df_me5a_2022_2025["Дата деблокирования"] = pd.to_datetime(df_me5a_2022_2025["Дата деблокирования"], format="%d.%m.%Y", errors="coerce").dt.date


In [14]:
df_me5a_cleaned =df_me5a_2022_2025.copy()

In [15]:
#me2n
df_me2n_2022_2025["﻿Документ закупки"] = df_me2n_2022_2025["﻿Документ закупки"].apply(temizle_ve_cevir)
df_me2n_2022_2025.rename(columns={'﻿Документ закупки': 'Документ закупки'}, inplace=True)
df_me2n_2022_2025["Дата документа"] = pd.to_datetime(df_me2n_2022_2025["Дата документа"], format="%d.%m.%Y", errors="coerce").dt.date
df_me2n_2022_2025["Заявка"] = df_me2n_2022_2025["Заявка"].apply(temizle_ve_cevir)
df_me2n_2022_2025["Материал"] = df_me2n_2022_2025["Материал"].apply(temizle_ve_cevir)
df_me2n_2022_2025 = duzenle_sayi_sutunu(df_me2n_2022_2025, "Объем заказа")
df_me2n_2022_2025 = duzenle_sayi_sutunu(df_me2n_2022_2025, "еще поставить (количество)")
df_me2n_2022_2025 = duzenle_sayi_sutunu(df_me2n_2022_2025, "Количество в СЕИ")
df_me2n_2022_2025 = duzenle_sayi_sutunu(df_me2n_2022_2025, "Цена нетто")
df_me2n_2022_2025 = duzenle_sayi_sutunu(df_me2n_2022_2025, "СтоимЗаказа нетто")
df_me2n_2022_2025 = duzenle_sayi_sutunu(df_me2n_2022_2025, "Еще для поставки (стоимость)")
df_me2n_2022_2025["Дата поставки"] = pd.to_datetime(df_me2n_2022_2025["Дата поставки"], format="%d.%m.%Y", errors="coerce").dt.date


In [16]:
#zmb51
birlesik_df_zmb51 = duzenle_sayi_sutunu(birlesik_df_zmb51, "Сумма во ВВ")
birlesik_df_zmb51["Партия"] = birlesik_df_zmb51["Партия"].apply(temizle_ve_cevir)
birlesik_df_zmb51["Материал"] = birlesik_df_zmb51["Материал"].apply(temizle_ve_cevir)
birlesik_df_zmb51["Дата ввода"] = pd.to_datetime(birlesik_df_zmb51["Дата ввода"], format="%d.%m.%Y", errors="coerce").dt.date
birlesik_df_zmb51["Дата документа"] = pd.to_datetime(birlesik_df_zmb51["Дата документа"], format="%d.%m.%Y", errors="coerce").dt.date
birlesik_df_zmb51["Дата проводки"] = pd.to_datetime(birlesik_df_zmb51["Дата проводки"], format="%d.%m.%Y", errors="coerce").dt.date
birlesik_df_zmb51["Внутренний заказ"] = birlesik_df_zmb51["Внутренний заказ"].apply(temizle_ve_cevir)
birlesik_df_zmb51["Заказ на поставку"] = birlesik_df_zmb51["Заказ на поставку"].apply(temizle_ve_cevir)
birlesik_df_zmb51["Позиция"] = birlesik_df_zmb51["Позиция"].apply(temizle_ve_cevir)
birlesik_df_zmb51["Документ материала"] = birlesik_df_zmb51["Документ материала"].apply(temizle_ve_cevir)
birlesik_df_zmb51["Поз. док. материала"] = birlesik_df_zmb51["Поз. док. материала"].apply(temizle_ve_cevir)
birlesik_df_zmb51 = duzenle_sayi_sutunu(birlesik_df_zmb51, "Кол-во в ЕИ ввода")
birlesik_df_zmb51 = duzenle_sayi_sutunu(birlesik_df_zmb51, "Количество")
birlesik_df_zmb51['Поставщик'] = birlesik_df_zmb51['Поставщик'].apply(temizle_ve_cevir)
birlesik_df_zmb51.rename(columns={'﻿Вид движения': 'Вид движения'}, inplace=True)


In [17]:
#zmm067
df_zmm067_2022_2025["﻿Заявка"] = df_zmm067_2022_2025["﻿Заявка"].apply(temizle_ve_cevir)
df_zmm067_2022_2025.rename(columns={'﻿Заявка': 'Заявка'}, inplace=True)
df_zmm067_2022_2025["Позиция заявки"] = df_zmm067_2022_2025["Позиция заявки"].apply(temizle_ve_cevir)
df_zmm067_2022_2025["Материал"] = df_zmm067_2022_2025["Материал"].apply(temizle_ve_cevir)
df_zmm067_2022_2025["Заказ на поставку"] = df_zmm067_2022_2025["Заказ на поставку"].apply(temizle_ve_cevir)
df_zmm067_2022_2025["Позиция ЗкзНаПостав"] = df_zmm067_2022_2025["Позиция ЗкзНаПостав"].apply(temizle_ve_cevir)



In [18]:
#df_zmm067_2022_2025["Позиция ЗкзНаПостав"] = df_zmm067_2022_2025["Позиция ЗкзНаПостав"].apply(temizle_ve_cevir)


In [19]:
#df_zmm067_2022_2025["Дополнительное"].sample(n=5)

In [20]:
#df_zmm067_2022_2025.columns

In [21]:
df_me5a_2022_2025 = df_me5a_2022_2025.map(temizle_nan)
df_zmm059_2022_2025 = df_zmm059_2022_2025.map(temizle_nan)
df_me2n_2022_2025 = df_me2n_2022_2025.map(temizle_nan)
df_tedarikci_listesi = df_tedarikci_listesi.map(temizle_nan)
birlesik_df_zmb51 = birlesik_df_zmb51.map(temizle_nan)
df_zmm067_2022_2025 = df_zmm067_2022_2025.map(temizle_nan)

In [22]:
df_calisilan = df_zmm059_2022_2025.copy()

In [23]:
df_calisilan["B3-B3POZ"] = (
    df_calisilan["Заявка"].astype(str) + "_" + df_calisilan["Позиция заявки"].astype(str)
)
df_me5a_2022_2025["B3-B3POZ"] = (
    df_me5a_2022_2025["Заявка"].astype(str) + "_" + df_me5a_2022_2025["Позиция заявки"].astype(str)
)

In [24]:
df_calisilan.columns.values[2] = "Зп_Поз"
df_tedarikci_listesi.rename(columns={'kod': 'Поставщик'}, inplace=True)

In [25]:
adetler = df_calisilan["B3-B3POZ"].value_counts()

In [26]:
mukerrer_kodlar = adetler[adetler > 1].index

In [27]:
silinmesi_gerekenler = df_calisilan[
    (df_calisilan["B3-B3POZ"].isin(mukerrer_kodlar)) &
    (df_calisilan["Документ_закупки"].isna())

]

In [28]:
df_calisilan = df_calisilan.drop(silinmesi_gerekenler.index)

In [29]:
#df_calisilan["Поставщик"] = df_calisilan["Поставщик"].apply(temizle_ve_cevir)

In [30]:
# Tedarikçi sözlüğünü yine string anahtarlara göre oluştur
tedarikci_dict = dict(zip(df_tedarikci_listesi.iloc[:, 0].astype(str), df_tedarikci_listesi.iloc[:, 1]))

# Eşleştirme
df_calisilan["имя_поставщика"] = df_calisilan["Поставщик"].map(tedarikci_dict)

In [31]:
#df_calisilan.tail()

In [32]:
# Sözlük oluştur (anahtar: B3-B3POZ, değer: Материал)
b3_to_material_dict = dict(zip(df_me5a_2022_2025["B3-B3POZ"].astype(str), df_me5a_2022_2025["Материал"]))

In [33]:
# B3-B3POZ'e göre eşleşen Материал'ları getir (yeni bir sütun olarak)
df_calisilan["Материал_yeni"] = df_calisilan["B3-B3POZ"].astype(str).map(b3_to_material_dict)

In [34]:
# Sadece boş olan Материал hücrelerini, eşleşen değerle doldur
df_calisilan["Материал"] = df_calisilan["Материал"].fillna(df_calisilan["Материал_yeni"])
# Geçici sütunu silelim
df_calisilan.drop(columns=["Материал_yeni"], inplace=True)

In [35]:
# Sözlükleri oluştur
ru_dict = dict(zip(df_material_list.iloc[:, 0].astype(str), df_material_list.iloc[:, 1]))
tr_dict = dict(zip(df_material_list.iloc[:, 0].astype(str), df_material_list.iloc[:, 2]))

# Eşleştir
df_calisilan["RU Наименование"] = df_calisilan["Материал"].astype(str).map(ru_dict)
df_calisilan["TR Наименование"] = df_calisilan["Материал"].astype(str).map(tr_dict)


In [36]:
#df_calisilan["Материал"].isna().sum()

In [37]:
#df_calisilan[["Поставщик", "имя_поставщика"]].sample(n=5)

In [38]:
df_calisilan.drop(columns=['Материалы тур.', 'Материалы анг.'], inplace=True)

In [39]:
# Sütunları çıkart
col_ru = df_calisilan.pop('RU Наименование')
col_tr = df_calisilan.pop('TR Наименование')
col_b3 = df_calisilan.pop('B3-B3POZ')

# Sütunları istediğin yerlere ekle
df_calisilan.insert(1, 'B3-B3POZ', col_b3)
df_calisilan.insert(8, 'RU Наименование', col_ru)
df_calisilan.insert(9, 'TR Наименование', col_tr)


In [40]:
 # Sözlük oluştur (anahtar: B3-B3POZ, değer: Материал)
b3_to_material_dict = dict(zip(df_me5a_2022_2025["B3-B3POZ"].astype(str), df_me5a_2022_2025["Материал"]))

In [41]:
# B3-B3POZ'e göre eşleşen Материал'ları getir (yeni bir sütun olarak)
df_calisilan["Материал_yeni"] = df_calisilan["B3-B3POZ"].astype(str).map(b3_to_material_dict)

In [42]:
# Sadece boş olan Материал hücrelerini, eşleşen değerle doldur
df_calisilan["Материал"] = df_calisilan["Материал"].fillna(df_calisilan["Материал_yeni"])
# Geçici sütunu silelim
df_calisilan.drop(columns=["Материал_yeni"], inplace=True)

In [43]:
# Sözlükleri oluştur
ru_dict = dict(zip(df_material_list.iloc[:, 0].astype(str), df_material_list.iloc[:, 1]))
tr_dict = dict(zip(df_material_list.iloc[:, 0].astype(str), df_material_list.iloc[:, 2]))

# Eşleştir
df_calisilan["RU Наименование"] = df_calisilan["Материал"].astype(str).map(ru_dict)
df_calisilan["TR Наименование"] = df_calisilan["Материал"].astype(str).map(tr_dict)

In [44]:
df_calisilan["Материал"].isna().sum()

np.int64(5038)

In [45]:
df_calisilan = df_calisilan.map(temizle_nan)

In [46]:
sorted_columns = sorted(df_calisilan.columns)
print(sorted_columns)

['B3-B3POZ', 'RU Наименование', 'TR Наименование', 'Автор Внутреннего заказа', 'БЕ', 'Валюта', 'Валюта ЗП', 'Валюта.1', 'Валюта.2', 'Вид докум. закупки', 'Вид докум. закупки.1', 'ГОД', 'Группа закупок', 'Группа закупок.1', 'Группа материалов', 'Дата вступления в силу контракта', 'Дата доставки, одобренная Поставщиком', 'Дата контракта', 'Дата окончания действия договора', 'Дата поставки', 'Дата проводки', 'Дата создания', 'Дата создания.1', 'Документ_закупки', 'ЕИ заказа на постав.', 'Единица ЗП', 'Единица цены', 'Завод', 'Закуп. организация', 'Заявка', 'Значок "поставка завершена"', 'Зп_Поз', 'Имя 1', 'Инвентарный номер в SAS', 'Индикатор удаления', 'Итого цена нетто', 'Количество ЗП', 'Количество по счету', 'Количество по счету.1', 'Количество по счету.2', 'Количество поставки', 'Краткий текст', 'Материал', 'Минимальная цена ЗП', 'Минимальная цена ЗП.1', 'Минимальная цена ЗП.2', 'Название ГрЗакупок', 'Название ГрЗакупок.1', 'Название ЗакупОрг', 'Название группы', 'Название фирмы', 'Н

In [47]:
#df_calisilan[["Документ_закупки","Зп_Поз","Количество ЗП","Цена нетто ЗП","Итого цена нетто"]].sample(n=5, random_state=1)

In [48]:
# Hepsini string yap ve boşlukları temizle
birlesik_df_zmb51["Заказ на поставку"] = birlesik_df_zmb51["Заказ на поставку"].astype(str).str.strip()
df_me2n_2022_2025["Документ закупки"] = df_me2n_2022_2025["Документ закупки"].astype(str).str.strip()


In [49]:
# "Наш знак" için dict
dict_nash_znak = dict(zip(df_me2n_2022_2025["Документ закупки"], df_me2n_2022_2025["Наш знак"]))

# "Ваш код" için dict
dict_vash_kod = dict(zip(df_me2n_2022_2025["Документ закупки"], df_me2n_2022_2025["Ваш код"]))

# map ile ekleme
birlesik_df_zmb51["Наш знак"] = birlesik_df_zmb51["Заказ на поставку"].map(dict_nash_znak)
birlesik_df_zmb51["Ваш код"] = birlesik_df_zmb51["Заказ на поставку"].map(dict_vash_kod)

In [50]:
df_calisilan["Позиция заявки"].apply(temizle_ve_cevir)

,Позиция заявки
0,10
1,10
2,10
3,10
4,20
...,...
767882,<NA>
767883,<NA>
767884,<NA>
767885,<NA>


In [51]:
# 1. Anahtar sütunlarını birleştir ve zp_zppoz adını ver
birlesik_df_zmb51["zp_zppoz"] = (
    birlesik_df_zmb51["Заказ на поставку"].apply(temizle_ve_cevir) + "_" +
    birlesik_df_zmb51["Позиция"].apply(temizle_ve_cevir)
)

df_calisilan["zp_zppoz"] = (
    df_calisilan["Документ_закупки"].apply(temizle_ve_cevir) + "_" +
    df_calisilan["Зп_Поз"].apply(temizle_ve_cevir)
)

# 2. dict oluştur
dict_poz = dict(zip(df_calisilan["zp_zppoz"], df_calisilan["Позиция заявки"]))

# 3. map ile yeni sütunu ekle
birlesik_df_zmb51["Позиция заявки"] = birlesik_df_zmb51["zp_zppoz"].map(dict_poz)
birlesik_df_zmb51["Позиция заявки"].apply(temizle_ve_cevir)
birlesik_df_zmb51["Позиция заявки"] = birlesik_df_zmb51["Позиция заявки"].fillna(0).astype(int)

/tmp/ipython-input-873327287.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  birlesik_df_zmb51["Позиция заявки"] = birlesik_df_zmb51["Позиция заявки"].fillna(0).astype(int)


In [52]:
# 1. Zorunlu temizleme işlemleri (orijinal sütunların üzerine yazar)
df_me5a_2022_2025["Заявка"] = df_me5a_2022_2025["Заявка"].apply(temizle_ve_cevir)
df_me5a_2022_2025["Позиция заявки"] = df_me5a_2022_2025["Позиция заявки"].apply(temizle_ve_cevir)

df_zmm067_2022_2025["Заявка"] = df_zmm067_2022_2025["Заявка"].apply(temizle_ve_cevir)
df_zmm067_2022_2025["Позиция заявки"] = df_zmm067_2022_2025["Позиция заявки"].apply(temizle_ve_cevir)

# 2. Anahtar oluştur
df_me5a_2022_2025["zayavka_poz"] = df_me5a_2022_2025["Заявка"] + "_" + df_me5a_2022_2025["Позиция заявки"]
df_zmm067_2022_2025["zayavka_poz"] = df_zmm067_2022_2025["Заявка"] + "_" + df_zmm067_2022_2025["Позиция заявки"]

# 3. Dict oluştur
dict_dop = dict(zip(df_zmm067_2022_2025["zayavka_poz"], df_zmm067_2022_2025["Дополнительное"]))

# 4. map ile yeni sütun ekle
df_me5a_2022_2025["Дополнительное"] = df_me5a_2022_2025["zayavka_poz"].map(dict_dop)

#yeni oluşturulan sutunlara ihtiyaç kalmadı

df_me5a_2022_2025 = df_me5a_2022_2025.drop(columns=["zayavka_poz"])
df_zmm067_2022_2025 = df_zmm067_2022_2025.drop(columns=["zayavka_poz"])

In [53]:
# 1. Sütunları temizle (aynı sütunların üzerine yazıyoruz)
df_me2n_2022_2025["Документ закупки"] = df_me2n_2022_2025["Документ закупки"].apply(temizle_ve_cevir)
df_me2n_2022_2025["Позиция"] = df_me2n_2022_2025["Позиция"].apply(temizle_ve_cevir)

df_calisilan["Документ_закупки"] = df_calisilan["Документ_закупки"].apply(temizle_ve_cevir)
df_calisilan["Зп_Поз"] = df_calisilan["Зп_Поз"].apply(temizle_ve_cevir)

# 2. Anahtar oluştur
df_me2n_2022_2025["doc_poz"] = df_me2n_2022_2025["Документ закупки"] + "_" + df_me2n_2022_2025["Позиция"]
df_calisilan["doc_poz"] = df_calisilan["Документ_закупки"] + "_" + df_calisilan["Зп_Поз"]

# 3. Dict oluştur (anahtar: doc_poz, değer: Создал)
dict_sozdal = dict(zip(df_calisilan["doc_poz"], df_calisilan["Создал"]))

# 4. map ile df_me2n'e ekle
df_me2n_2022_2025["Создал"] = df_me2n_2022_2025["doc_poz"].map(dict_sozdal)

# yeni oluşturulan sütunlara ihtiyaç kalmadı
df_me2n_2022_2025 = df_me2n_2022_2025.drop(columns=["doc_poz"])
df_calisilan = df_calisilan.drop(columns=["doc_poz"])

In [54]:
# 1. Temizleme işlemi
birlesik_df_zmb51["Поставщик"] = birlesik_df_zmb51["Поставщик"].apply(temizle_ve_cevir)
df_tedarikci_listesi["﻿Кредитор"] = df_tedarikci_listesi["﻿Кредитор"].apply(temizle_ve_cevir)

# 2. Anahtar oluştur (tek sütun olduğu için direkt kullanabiliriz)
# 3. Dict oluştur
dict_suppliers = dict(zip(df_tedarikci_listesi["﻿Кредитор"], df_tedarikci_listesi["Имя поставщика"]))

# 4. map ile birlesik_df_zmb51'e ekle
birlesik_df_zmb51["Имя поставщика"] = birlesik_df_zmb51["Поставщик"].map(dict_suppliers)


In [55]:
# 1. Temizleme işlemi
birlesik_df_zmb51["Заказ на поставку"] = birlesik_df_zmb51["Заказ на поставку"].apply(temizle_ve_cevir)
birlesik_df_zmb51["Позиция"] = birlesik_df_zmb51["Позиция"].apply(temizle_ve_cevir)

df_me2n_2022_2025["Документ закупки"] = df_me2n_2022_2025["Документ закупки"].apply(temizle_ve_cevir)
df_me2n_2022_2025["Позиция"] = df_me2n_2022_2025["Позиция"].apply(temizle_ve_cevir)

# 2. Anahtar oluştur
birlesik_df_zmb51["doc_poz"] = birlesik_df_zmb51["Заказ на поставку"] + "_" + birlesik_df_zmb51["Позиция"]
df_me2n_2022_2025["doc_poz"] = df_me2n_2022_2025["Документ закупки"] + "_" + df_me2n_2022_2025["Позиция"]

# 3. Dict oluştur (anahtar: doc_poz, değer: Объем заказа)
dict_obem = dict(zip(df_me2n_2022_2025["doc_poz"], df_me2n_2022_2025["Объем заказа"]))

# 4. map ile birlesik_df_zmb51'e ekle
birlesik_df_zmb51["Объем заказа"] = birlesik_df_zmb51["doc_poz"].map(dict_obem)
#yeni oluşturulan sutunlara ihtiyaç kalmadı
birlesik_df_zmb51 = birlesik_df_zmb51.drop(columns=["doc_poz"])
df_me2n_2022_2025 = df_me2n_2022_2025.drop(columns=["doc_poz"])



In [56]:
df_me2n_2022_2025["Объем заказа"] = pd.to_numeric(df_me2n_2022_2025["Объем заказа"], errors="coerce")
df_me2n_2022_2025["еще поставить (количество)"] = pd.to_numeric(
    df_me2n_2022_2025["еще поставить (количество)"].fillna(0), errors="coerce"
)

# 2. Hesaplama
df_me2n_2022_2025["Уже поставлено (количество)"] = (
    df_me2n_2022_2025["Объем заказа"] - df_me2n_2022_2025["еще поставить (количество)"]
)

In [57]:
# 1. Temizleme işlemi
df_me2n_2022_2025["Документ закупки"] = df_me2n_2022_2025["Документ закупки"].apply(temizle_ve_cevir)
df_me2n_2022_2025["Позиция"] = df_me2n_2022_2025["Позиция"].apply(temizle_ve_cevir)


df_calisilan["Документ_закупки"] = df_calisilan["Документ_закупки"].apply(temizle_ve_cevir)
df_calisilan["Зп_Поз"] = df_calisilan["Зп_Поз"].apply(temizle_ve_cevir)

# 2. Anahtar oluştur
df_me2n_2022_2025["doc_poz"] = df_me2n_2022_2025["Документ закупки"] + "_" + df_me2n_2022_2025["Позиция"]
df_calisilan["doc_poz"] = df_calisilan["Документ_закупки"] + "_" + df_calisilan["Зп_Поз"]

# 3. Dict oluştur (anahtar: doc_poz, değer: Позиция заявки)
dict_poz = dict(zip(df_calisilan["doc_poz"], df_calisilan["Позиция заявки"]))

# 4. map ile df_me2n_2022_2025'e ekle
df_me2n_2022_2025["Позиция заявки"] = df_me2n_2022_2025["doc_poz"].map(dict_poz)
#yeni oluşturulan sutunlara ihtiyaç kalmadı
df_calisilan = df_calisilan.drop(columns=["doc_poz"])
df_me2n_2022_2025 = df_me2n_2022_2025.drop(columns=["doc_poz"])


In [58]:
df_me2n_2022_2025["Материал_ME2N"] = df_me2n_2022_2025["Материал"]
df_me5a_2022_2025["Материал_ME5A"] = df_me5a_2022_2025["Материал"]
# 1. Temizleme işlemi
df_me2n_2022_2025["Заявка"] = df_me2n_2022_2025["Заявка"].apply(temizle_ve_cevir)
df_me2n_2022_2025["Позиция заявки"] = df_me2n_2022_2025["Позиция заявки"].apply(temizle_ve_cevir)

df_me5a_2022_2025["Заявка"] = df_me5a_2022_2025["Заявка"].apply(temizle_ve_cevir)
df_me5a_2022_2025["Позиция заявки"] = df_me5a_2022_2025["Позиция заявки"].apply(temizle_ve_cevir)

# 2. Anahtar oluştur
df_me2n_2022_2025["zayavka_poz"] = df_me2n_2022_2025["Заявка"] + "_" + df_me2n_2022_2025["Позиция заявки"]
df_me5a_2022_2025["zayavka_poz"] = df_me5a_2022_2025["Заявка"] + "_" + df_me5a_2022_2025["Позиция заявки"]

# 3. Dict oluştur (anahtar: zayavka_poz, değer: Материал_ME5A)
dict_me5a = dict(zip(df_me5a_2022_2025["zayavka_poz"], df_me5a_2022_2025["Материал_ME5A"]))

# 4. map ile df_me2n_2022_2025'e ekle
df_me2n_2022_2025["Материал_ME5A"] = df_me2n_2022_2025["zayavka_poz"].map(dict_me5a)
#yeni oluşturulan sutunlara ihtiyaç kalmadı
df_me5a_2022_2025 = df_me5a_2022_2025.drop(columns=["zayavka_poz"])
df_me2n_2022_2025 = df_me2n_2022_2025.drop(columns=["zayavka_poz"])


In [59]:
df_me2n_2022_2025 = df_me2n_2022_2025.applymap(temizle_nan)

/tmp/ipython-input-2153232079.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_me2n_2022_2025 = df_me2n_2022_2025.applymap(temizle_nan)


In [60]:
df_me2n_2022_2025_analog = df_me2n_2022_2025[
    (df_me2n_2022_2025["Материал_ME2N"] != df_me2n_2022_2025["Материал_ME5A"]) &
    (df_me2n_2022_2025["Заявка"].apply(dolu_mi) & df_me2n_2022_2025["Позиция заявки"].apply(dolu_mi))
].copy()


In [61]:
# 1. Temizleme işlemi (gerekiyorsa)
df_me2n_2022_2025_analog["Материал_ME5A"] = df_me2n_2022_2025_analog["Материал_ME5A"].apply(temizle_ve_cevir)
df_material_list["﻿Материал"] = df_material_list["﻿Материал"].apply(temizle_ve_cevir)

# 2. Dict oluştur
dict_tr = dict(zip(df_material_list["﻿Материал"], df_material_list["TR Наименование"]))
dict_ru = dict(zip(df_material_list["﻿Материал"], df_material_list["RU Наименование"]))

# 3. map ile yeni sütunları ekle
df_me2n_2022_2025_analog["TR Наименование_ВЗ"] = df_me2n_2022_2025_analog["Материал_ME5A"].map(dict_tr)
df_me2n_2022_2025_analog["RU Наименование_ВЗ"] = df_me2n_2022_2025_analog["Материал_ME5A"].map(dict_ru)


In [62]:
df_me2n_2022_2025_analog = df_me2n_2022_2025_analog.rename(columns={"Наименование RU": "Наименование RU_ЗП"})
df_me2n_2022_2025_analog = df_me2n_2022_2025_analog.rename(columns={"Наименование TR": "Наименование TR_ЗП"})


In [63]:
# 1. Anahtar sütunları tuple haline getirelim
df_me2n_2022_2025["key"] = df_me2n_2022_2025[["Документ закупки", "Позиция"]].apply(tuple, axis=1)
df_me5a_2022_2025["key"] = df_me5a_2022_2025[["Заказ на поставку", "Позиция ЗкзНаПостав"]].apply(tuple, axis=1)

# 2. Aktarılacak sütunlar ve hedef adları
map_dict = {
    "Объем заказа": "Объем заказа",
    "еще поставить (количество)": "еще поставить (количество)",
    "Еще для поставки (стоимость)": "Еще для поставки (стоимость)",
    "Для фактурирования (колич.)": "Для фактурирования (колич.)",
    "Для фактурирования (стоим.)": "Для фактурирования (стоим.)",
    "Имя поставщика": "Имя поставщика",
    "Создал": "ЗП_Создал",
    "Наш знак":"Наш знак",
    "Цена нетто":"Цена нетто",
    "Валюта":"Валюта",
    "Индикатор удаления":"Индикатор удаления_зп",
    "ЕИ заказа на постав.":"ЕИ заказа на постав.",
    "Единица цены":"Единица цены",

}

# 3. Her sütun için dict map uygulama
for source_col, target_col in map_dict.items():
    temp_dict = dict(zip(df_me2n_2022_2025["key"], df_me2n_2022_2025[source_col]))
    df_me5a_2022_2025[target_col] = df_me5a_2022_2025["key"].map(temp_dict)

# 4. Geçici key sütunlarını kaldır
df_me2n_2022_2025.drop(columns=["key"], inplace=True)
df_me5a_2022_2025.drop(columns=["key"], inplace=True)


In [64]:
# 1. Yeni sütun sırasını belirle
df_me2n_2022_2025_analog_yeni_sira = [
    'ГОД', 'Документ закупки', 'Позиция', 'Группа закупок',
    'Закуп. организация','Создал', 'Индикатор удаления', 'Дата документа',
    'Наш знак', 'Ваш код', 'Имя поставщика', 'Заявка',  'Позиция заявки',
    'Материал_ME2N','Материал_ME5A','Наименование RU_ЗП','Наименование TR_ЗП',
    'RU Наименование_ВЗ','TR Наименование_ВЗ',
    'Объем заказа', 'еще поставить (количество)','Уже поставлено (количество)',
    'Складская ЕИ','Количество в СЕИ','Единица цены','Цена нетто',
    'СтоимЗаказа нетто','Еще для поставки (стоимость)','Валюта',
    'Вид докум. закупки','Код налога','Дата поставки','ЕИ заказа на постав.',
    'Инвентарный номер'
]

# 2. DataFrame'i yeni sütun sırasına göre düzenle
df_me2n_2022_2025_analog =df_me2n_2022_2025_analog[df_me2n_2022_2025_analog_yeni_sira]


In [65]:
# 1. Yeni sütun sırasını belirle
df_me2n_2022_2025_yeni_sira = [
      'ГОД', 'Документ закупки', 'Позиция','Создал', 'Группа закупок',
      'Закуп. организация', 'Индикатор удаления', 'Дата документа',
      'Наш знак', 'Ваш код', 'Имя поставщика', 'Заявка','Позиция заявки',
      'Материал','Инвентарный номер', 'Наименование RU','Наименование TR',
      'Объем заказа','Складская ЕИ','еще поставить (количество)',
      'Еще для поставки (стоимость)','Уже поставлено (количество)',
      'Для фактурирования (стоим.)','Для фактурирования (колич.)',
      'Количество в СЕИ', 'Единица цены','СтоимЗаказа нетто','Валюта',
      'Цена нетто','Вид докум. закупки', 'Код налога','Краткий текст',
      'Дата поставки', 'ЕИ заказа на постав.','ТипДокумЗакупки',
      'Тип неполноты данных','Группа материалов','Материал_ME2N', 'Материал_ME5A'

]

# 2. DataFrame'i yeni sütun sırasına göre düzenle
df_me2n_2022_2025 =df_me2n_2022_2025[df_me2n_2022_2025_yeni_sira]

In [66]:
df_me5a_2022_2025.rename(columns={"'Дата заявки": "Дата_заявки"}, inplace=True)


In [67]:
df_me5a_2022_2025_yeni_sira = [
       'ГОД', 'Заявка', 'Позиция заявки', 'Материал', 'Индикатор удаления_B3','ЗатребованКолич',
       'Объем заказа',  'Заказ на поставку', 'Позиция ЗкзНаПостав','Индикатор удаления_зп',
       'Затребовал', 'Индик. выдачи', 'Группа деблокир.',
       'Статус обработки заявки', 'Дата заявки', 'Группа закупок',
       'Дата изменения','Дата заказа', 'Долгосрочный договор',
       'Заказанное к-во', 'Единица измерения', 'RU Наименование',
       'TR Наименование',  'Дополнительное','Инвентарный номер', 'Краткий текст',
       'Закуп. организация','еще поставить (количество)','Еще для поставки (стоимость)',
       'Для фактурирования (колич.)','Для фактурирования (стоим.)', 'Имя поставщика',
       'ЗП_Создал','Наш знак', 'Цена нетто', 'Валюта', 'ЕИ заказа на постав.',
       'Единица цены', 'Создал', 'ПоступлМатериала', 'Поступление счета',
       'Номер проекта', 'Группа материалов', 'Дата поставки',
       'Завод-поставщик', 'Вид документа', 'Статус обработки',
       'Индикатор создания', 'Стратегия деблокир.', 'НДП-материал',
       'Номер потребности', 'Объем дефицита', 'Тип даты поставки',
       'Дата деблокирования', 'Дата_заявки', 'B3-B3POZ','Материал_ME5A'

]

# 2. DataFrame'i yeni sütun sırasına göre düzenle
df_me5a_2022_2025 =df_me5a_2022_2025[df_me5a_2022_2025_yeni_sira]

In [68]:
df_calisilan_yeni_sira = [
    'ГОД','zp_zppoz','B3-B3POZ','Документ_закупки', 'Зп_Поз',
    'Индикатор удаления','неполн.', 'Материал', 'Заявка', 'Позиция заявки',
    'RU Наименование', 'TR Наименование',
    'Инвентарный номер в SAS', 'Количество ЗП',
    'Минимальная цена ЗП', 'Минимальная цена ЗП.1',
    'Минимальная цена ЗП.2','Единица ЗП', 'Цена нетто ЗП', 'Итого цена нетто',
    'Валюта ЗП','Вид докум. закупки', 'Дата создания', 'Дата поставки',
    'Дата доставки, одобренная Поставщиком', 'Группа закупок',
    'Название ГрЗакупок', 'БЕ', 'Название фирмы', 'Закуп. организация',
    'Название ЗакупОрг', 'Завод', 'Имя 1', 'Поставщик', 'Группа материалов',
    'Название группы', 'Краткий текст', 'Создал', 'Вид докум. закупки.1',
    'Обозначение ВидДокум', 'Дата создания.1', 'Создал.1','Группа закупок.1',
    'Название ГрЗакупок.1', 'ЕИ заказа на постав.','Единица цены',
    'Цена заказа нетто', 'Стоимость брутто', 'Валюта','Недопоставленное количество',
    'Количество поставки', 'Сумма брутто','Валюта.1', 'Количество по счету', 'Дата проводки',
    'Количество по счету.1', 'Валюта.2', 'Остаток количества по счету',
    'Остаток суммы по счету', 'Автор Внутреннего заказа', 'Количество по счету.2',
    'Значок "поставка завершена"','Дата контракта', 'Дата вступления в силу контракта',
    'Дата окончания действия договора', 'Склад', '№ комиссии','имя_поставщика'
]

# 2. DataFrame'i yeni sütun sırasına göre düzenle
df_calisilan =df_calisilan[df_calisilan_yeni_sira]

In [69]:
# 1. Yeni sütun sırasını belirle
zmb51_yeni_sira = [
    'ГОД', 'Вид движения', 'Партия', 'Материал', 'Заказ на поставку',
    'Позиция', 'Наш знак', 'Ваш код', 'Внутренний заказ', 'Позиция заявки',
    'Склад', 'Дата ввода', 'Время ввода', 'Дата документа', 'Дата проводки',
    'Документ материала', 'Поз. док. материала', 'Признак 2', 'Ссылка',
    'Краткий текст материала', 'Кол-во в ЕИ ввода', 'Объем заказа',
    'ЕИ ввода','Сумма во ВВ', 'Количество', 'Поставщик','Имя поставщика',
    'Накладная','Текст заголовка документа', 'Заявитель', 'Инвентарный номер',
    'Имя пользователя', 'БЕ', 'Базовая ЕИ', 'ГодДокумМатериала', 'Валюта',
    'Завод', 'Имя 1', 'Основное средство', 'ЕИ цены заказа','ЕИ заказа на постав.'

]

# 2. DataFrame'i yeni sütun sırasına göre düzenle
birlesik_df_zmb51 = birlesik_df_zmb51[zmb51_yeni_sira]

In [70]:
#ME5A
df_me5a_2022_2025["ГОД"] = pd.to_numeric(df_me5a_2022_2025["ГОД"], errors="coerce").astype("Int64")
df_me5a_2022_2025["Заявка"] = pd.to_numeric(df_me5a_2022_2025["Заявка"], errors="coerce").astype("Int64")
df_me5a_2022_2025["Материал"] = pd.to_numeric(df_me5a_2022_2025["Материал"], errors="coerce").astype("Int64")
df_me5a_2022_2025["Заказ на поставку"] = pd.to_numeric(df_me5a_2022_2025["Заказ на поставку"], errors="coerce").astype("Int64")
df_me5a_2022_2025["Позиция ЗкзНаПостав"] = pd.to_numeric(df_me5a_2022_2025["Позиция ЗкзНаПостав"], errors="coerce").astype("Int64")
df_me5a_2022_2025["ЗатребованКолич"] = pd.to_numeric(df_me5a_2022_2025["ЗатребованКолич"], errors="coerce").astype("float64")
df_me5a_2022_2025["Заказанное к-во"] = pd.to_numeric(df_me5a_2022_2025["Заказанное к-во"], errors="coerce").astype("float64")
df_me5a_2022_2025["Объем заказа"] = pd.to_numeric(df_me5a_2022_2025["Объем заказа"], errors="coerce").astype("float64")
df_me5a_2022_2025["еще поставить (количество)"] = pd.to_numeric(df_me5a_2022_2025["еще поставить (количество)"], errors="coerce").astype("float64")
df_me5a_2022_2025["Еще для поставки (стоимость)"] = pd.to_numeric(df_me5a_2022_2025["Еще для поставки (стоимость)"], errors="coerce").astype("float64")
df_me5a_2022_2025["Для фактурирования (колич.)"] = pd.to_numeric(df_me5a_2022_2025["Для фактурирования (колич.)"], errors="coerce").astype("float64")
df_me5a_2022_2025["Для фактурирования (стоим.)"] = pd.to_numeric(df_me5a_2022_2025["Для фактурирования (стоим.)"], errors="coerce").astype("float64")
#ME2N
df_me2n_2022_2025["ГОД"] = pd.to_numeric(df_me2n_2022_2025["ГОД"], errors="coerce").astype("Int64")
df_me2n_2022_2025["Документ закупки"] = pd.to_numeric(df_me2n_2022_2025["Документ закупки"], errors="coerce").astype("Int64")
df_me2n_2022_2025["Позиция"] = pd.to_numeric(df_me2n_2022_2025["Позиция"], errors="coerce").astype("Int64")
df_me2n_2022_2025["Заявка"] = pd.to_numeric(df_me2n_2022_2025["Заявка"], errors="coerce").astype("Int64")
df_me2n_2022_2025["Материал"] = pd.to_numeric(df_me2n_2022_2025["Материал"], errors="coerce").astype("Int64")
df_me2n_2022_2025["Объем заказа"] = pd.to_numeric(df_me2n_2022_2025["Объем заказа"], errors="coerce").astype("float64")
df_me2n_2022_2025["еще поставить (количество)"] = pd.to_numeric(df_me2n_2022_2025["еще поставить (количество)"], errors="coerce").astype("float64")
df_me2n_2022_2025["Уже поставлено (количество)"] = pd.to_numeric(df_me2n_2022_2025["Уже поставлено (количество)"], errors="coerce").astype("float64")
df_me2n_2022_2025["Для фактурирования (стоим.)"] = pd.to_numeric(df_me2n_2022_2025["Для фактурирования (стоим.)"], errors="coerce").astype("float64")
df_me2n_2022_2025["Для фактурирования (колич.)"] = pd.to_numeric(df_me2n_2022_2025["Для фактурирования (колич.)"], errors="coerce").astype("float64")
df_me2n_2022_2025["Количество в СЕИ"] = pd.to_numeric(df_me2n_2022_2025["Количество в СЕИ"], errors="coerce").astype("float64")
df_me2n_2022_2025["Цена нетто"] = pd.to_numeric(df_me2n_2022_2025["Цена нетто"], errors="coerce").astype("float64")
df_me2n_2022_2025["СтоимЗаказа нетто"] = pd.to_numeric(df_me2n_2022_2025["СтоимЗаказа нетто"], errors="coerce").astype("float64")
df_me2n_2022_2025["Еще для поставки (стоимость)"] = pd.to_numeric(df_me2n_2022_2025["Еще для поставки (стоимость)"], errors="coerce").astype("float64")
#zmb51
birlesik_df_zmb51["ГОД"] = pd.to_numeric(birlesik_df_zmb51["ГОД"], errors="coerce").astype("Int64")
birlesik_df_zmb51["Партия"] = pd.to_numeric(birlesik_df_zmb51["Партия"], errors="coerce").astype("Int64")
birlesik_df_zmb51["Заказ на поставку"] = pd.to_numeric(birlesik_df_zmb51["Заказ на поставку"], errors="coerce").astype("Int64")
birlesik_df_zmb51["Позиция"] = pd.to_numeric(birlesik_df_zmb51["Позиция"], errors="coerce").astype("Int64")
birlesik_df_zmb51["Внутренний заказ"] = pd.to_numeric(birlesik_df_zmb51["Внутренний заказ"], errors="coerce").astype("Int64")
birlesik_df_zmb51["Документ материала"] = pd.to_numeric(birlesik_df_zmb51["Документ материала"], errors="coerce").astype("Int64")
birlesik_df_zmb51["Поз. док. материала"] = pd.to_numeric(birlesik_df_zmb51["Поз. док. материала"], errors="coerce").astype("Int64")
birlesik_df_zmb51["Кол-во в ЕИ ввода"] = pd.to_numeric(birlesik_df_zmb51["Кол-во в ЕИ ввода"], errors="coerce").astype("float64")
birlesik_df_zmb51["Сумма во ВВ"] = pd.to_numeric(birlesik_df_zmb51["Сумма во ВВ"], errors="coerce").astype("float64")
birlesik_df_zmb51["Количество"] = pd.to_numeric(birlesik_df_zmb51["Количество"], errors="coerce").astype("float64")
birlesik_df_zmb51["Поставщик"] = pd.to_numeric(birlesik_df_zmb51["Поставщик"], errors="coerce").astype("Int64")
#zmm059
df_calisilan["ГОД"] = pd.to_numeric(df_calisilan["ГОД"], errors="coerce").astype("Int64")
df_calisilan["Документ_закупки"] = pd.to_numeric(df_calisilan["Документ_закупки"], errors="coerce").astype("Int64")
df_calisilan["Зп_Поз"] = pd.to_numeric(df_calisilan["Зп_Поз"], errors="coerce").astype("Int64")
df_calisilan["Материал"] = pd.to_numeric(df_calisilan["Материал"], errors="coerce").astype("Int64")
df_calisilan["Заявка"] = pd.to_numeric(df_calisilan["Заявка"], errors="coerce").astype("Int64")
df_calisilan["Позиция заявки"] = pd.to_numeric(df_calisilan["Позиция заявки"], errors="coerce").astype("Int64")
df_calisilan["Количество ЗП"] = pd.to_numeric(df_calisilan["Количество ЗП"], errors="coerce").astype("float64")
df_calisilan["Минимальная цена ЗП"] = pd.to_numeric(df_calisilan["Минимальная цена ЗП"], errors="coerce").astype("float64")
df_calisilan["Минимальная цена ЗП.1"] = pd.to_numeric(df_calisilan["Минимальная цена ЗП.1"], errors="coerce").astype("float64")
df_calisilan["Единица ЗП"] = pd.to_numeric(df_calisilan["Единица ЗП"], errors="coerce").astype("Int64")
df_calisilan["Цена нетто ЗП"] = pd.to_numeric(df_calisilan["Цена нетто ЗП"], errors="coerce").astype("float64")
df_calisilan["Итого цена нетто"] = pd.to_numeric(df_calisilan["Итого цена нетто"], errors="coerce").astype("float64")
df_calisilan["Поставщик"] = pd.to_numeric(df_calisilan["Поставщик"], errors="coerce").astype("Int64")
#me2n_analog
df_me2n_2022_2025_analog["ГОД"] = pd.to_numeric(df_me2n_2022_2025["ГОД"], errors="coerce").astype("Int64")
df_me2n_2022_2025_analog["Документ закупки"] = pd.to_numeric(df_me2n_2022_2025["Документ закупки"], errors="coerce").astype("Int64")
df_me2n_2022_2025_analog["Заявка"] = pd.to_numeric(df_me2n_2022_2025["Заявка"], errors="coerce").astype("Int64")
df_me2n_2022_2025_analog["Материал"] = pd.to_numeric(df_me2n_2022_2025["Материал"], errors="coerce").astype("Int64")
df_me2n_2022_2025_analog["Объем заказа"] = pd.to_numeric(df_me2n_2022_2025["Объем заказа"], errors="coerce").astype("float64")
df_me2n_2022_2025_analog["еще поставить (количество)"] = pd.to_numeric(df_me2n_2022_2025["еще поставить (количество)"], errors="coerce").astype("float64")
df_me2n_2022_2025_analog["Количество в СЕИ"] = pd.to_numeric(df_me2n_2022_2025["Количество в СЕИ"], errors="coerce").astype("float64")
df_me2n_2022_2025_analog["Цена нетто"] = pd.to_numeric(df_me2n_2022_2025["Цена нетто"], errors="coerce").astype("float64")
df_me2n_2022_2025_analog["СтоимЗаказа нетто"] = pd.to_numeric(df_me2n_2022_2025["СтоимЗаказа нетто"], errors="coerce").astype("float64")
df_me2n_2022_2025_analog["Еще для поставки (стоимость)"] = pd.to_numeric(df_me2n_2022_2025["Еще для поставки (стоимость)"], errors="coerce").astype("float64")

#zmm067
df_zmm067_2022_2025.rename(columns={"SAT Yaratma Tarihi": "Дата Заявка"}, inplace=True)
df_zmm067_2022_2025.rename(columns={"Sipariş Silme Göstergesi": "Индикатор удаления заказа"}, inplace=True)
df_zmm067_2022_2025.rename(columns={"SAT Teslimat Tarihi": "Дата доставки заявка"}, inplace=True)
df_zmm067_2022_2025.rename(columns={"Talep Silme Göstergesi": "Индикатор удаления заявка"}, inplace=True)
df_zmm067_2022_2025["Объем заказа"] = pd.to_numeric(df_zmm067_2022_2025["Объем заказа"], errors="coerce").astype("float64")

In [71]:
df_calisilan_cleaned =df_calisilan.copy()

In [72]:
df_me5a_cleaned.columns

Index(['ГОД', 'Заявка', 'Позиция заявки', 'Материал', 'Индикатор удаления_B3',
       'Затребовал', 'Индик. выдачи', 'Группа деблокир.',
       'Статус обработки заявки', 'Дата заявки', 'Группа закупок',
       'Дата изменения', 'Заказ на поставку', 'Позиция ЗкзНаПостав',
       'Дата заказа', 'Долгосрочный договор', 'ЗатребованКолич',
       'Заказанное к-во', 'Единица измерения', 'RU Наименование',
       'TR Наименование', 'Инвентарный номер', 'Краткий текст',
       'Закуп. организация', 'Создал', 'ПоступлМатериала', 'Поступление счета',
       'Номер проекта', 'Группа материалов', 'Дата поставки',
       'Завод-поставщик', 'Вид документа', 'Статус обработки',
       'Индикатор создания', 'Стратегия деблокир.', 'НДП-материал',
       'Номер потребности', 'Объем дефицита', 'Тип даты поставки',
       'Дата деблокирования', ''Дата заявки'],
      dtype='object')

In [73]:
df_me5a_cleaned = df_me5a_cleaned[~df_me5a_cleaned["Индикатор удаления_B3"]]

In [74]:
# sadece NaN olanları tutar; boş string ('') varsa bu satırlar silinebilir
df_calisilan_cleaned = df_calisilan_cleaned[df_calisilan_cleaned['Индикатор удаления'].isna()]


In [75]:
df_calisilan_cleaned['Индикатор удаления']

,Индикатор удаления
0,<NA>
1,<NA>
2,<NA>
3,<NA>
4,<NA>
...,...
767882,<NA>
767883,<NA>
767884,<NA>
767885,<NA>


In [76]:
df_calisilan_cleaned.columns


Index(['ГОД', 'zp_zppoz', 'B3-B3POZ', 'Документ_закупки', 'Зп_Поз',
       'Индикатор удаления', 'неполн.', 'Материал', 'Заявка', 'Позиция заявки',
       'RU Наименование', 'TR Наименование', 'Инвентарный номер в SAS',
       'Количество ЗП', 'Минимальная цена ЗП', 'Минимальная цена ЗП.1',
       'Минимальная цена ЗП.2', 'Единица ЗП', 'Цена нетто ЗП',
       'Итого цена нетто', 'Валюта ЗП', 'Вид докум. закупки', 'Дата создания',
       'Дата поставки', 'Дата доставки, одобренная Поставщиком',
       'Группа закупок', 'Название ГрЗакупок', 'БЕ', 'Название фирмы',
       'Закуп. организация', 'Название ЗакупОрг', 'Завод', 'Имя 1',
       'Поставщик', 'Группа материалов', 'Название группы', 'Краткий текст',
       'Создал', 'Вид докум. закупки.1', 'Обозначение ВидДокум',
       'Дата создания.1', 'Создал.1', 'Группа закупок.1',
       'Название ГрЗакупок.1', 'ЕИ заказа на постав.', 'Единица цены',
       'Цена заказа нетто', 'Стоимость брутто', 'Валюта',
       'Недопоставленное количест

In [77]:
df_me5a_cleaned.columns

Index(['ГОД', 'Заявка', 'Позиция заявки', 'Материал', 'Индикатор удаления_B3',
       'Затребовал', 'Индик. выдачи', 'Группа деблокир.',
       'Статус обработки заявки', 'Дата заявки', 'Группа закупок',
       'Дата изменения', 'Заказ на поставку', 'Позиция ЗкзНаПостав',
       'Дата заказа', 'Долгосрочный договор', 'ЗатребованКолич',
       'Заказанное к-во', 'Единица измерения', 'RU Наименование',
       'TR Наименование', 'Инвентарный номер', 'Краткий текст',
       'Закуп. организация', 'Создал', 'ПоступлМатериала', 'Поступление счета',
       'Номер проекта', 'Группа материалов', 'Дата поставки',
       'Завод-поставщик', 'Вид документа', 'Статус обработки',
       'Индикатор создания', 'Стратегия деблокир.', 'НДП-материал',
       'Номер потребности', 'Объем дефицита', 'Тип даты поставки',
       'Дата деблокирования', ''Дата заявки'],
      dtype='object')

In [78]:
df_me5a_cleaned.loc[:, 'zp_zppoz'] = (
    df_me5a_cleaned['Заказ на поставку'].astype(str) + '_' +
    df_me5a_cleaned['Позиция ЗкзНаПостав'].astype(str)
)

In [79]:
df_me5a_cleaned.loc[:, 'B3-POZ'] = (
    df_me5a_cleaned['Заявка'].astype(str) + '_' +
    df_me5a_cleaned['Позиция заявки'].astype(str)
)

In [80]:
df_me5a_cleaned.loc[df_me5a_cleaned['Заказ на поставку'].notna(), 'Заказ на поставку'] = pd.NA
df_me5a_cleaned.loc[df_me5a_cleaned['Позиция ЗкзНаПостав'].notna(), 'Позиция ЗкзНаПостав'] = pd.NA

In [81]:
df_me5a_cleaned['ЗатребованКолич']

,ЗатребованКолич
1,1
2,1
3,1
4,1
5,1
...,...
387591,5
387592,170
387593,163
387594,2


In [82]:
df_me5a_cleaned.loc[:, "Заказанное к-во"] = pd.to_numeric(
    df_me5a_cleaned["Заказанное к-во"], errors="coerce"
).astype("float64")

In [83]:
df_me5a_cleaned.loc[:, "ЗатребованКолич"] = pd.to_numeric(
    df_me5a_cleaned["ЗатребованКолич"], errors="coerce"
).astype("float64")

In [84]:
df_me5a_cleaned.columns

Index(['ГОД', 'Заявка', 'Позиция заявки', 'Материал', 'Индикатор удаления_B3',
       'Затребовал', 'Индик. выдачи', 'Группа деблокир.',
       'Статус обработки заявки', 'Дата заявки', 'Группа закупок',
       'Дата изменения', 'Заказ на поставку', 'Позиция ЗкзНаПостав',
       'Дата заказа', 'Долгосрочный договор', 'ЗатребованКолич',
       'Заказанное к-во', 'Единица измерения', 'RU Наименование',
       'TR Наименование', 'Инвентарный номер', 'Краткий текст',
       'Закуп. организация', 'Создал', 'ПоступлМатериала', 'Поступление счета',
       'Номер проекта', 'Группа материалов', 'Дата поставки',
       'Завод-поставщик', 'Вид документа', 'Статус обработки',
       'Индикатор создания', 'Стратегия деблокир.', 'НДП-материал',
       'Номер потребности', 'Объем дефицита', 'Тип даты поставки',
       'Дата деблокирования', ''Дата заявки', 'zp_zppoz', 'B3-POZ'],
      dtype='object')

In [85]:
df_calisilan_cleaned.columns

Index(['ГОД', 'zp_zppoz', 'B3-B3POZ', 'Документ_закупки', 'Зп_Поз',
       'Индикатор удаления', 'неполн.', 'Материал', 'Заявка', 'Позиция заявки',
       'RU Наименование', 'TR Наименование', 'Инвентарный номер в SAS',
       'Количество ЗП', 'Минимальная цена ЗП', 'Минимальная цена ЗП.1',
       'Минимальная цена ЗП.2', 'Единица ЗП', 'Цена нетто ЗП',
       'Итого цена нетто', 'Валюта ЗП', 'Вид докум. закупки', 'Дата создания',
       'Дата поставки', 'Дата доставки, одобренная Поставщиком',
       'Группа закупок', 'Название ГрЗакупок', 'БЕ', 'Название фирмы',
       'Закуп. организация', 'Название ЗакупОрг', 'Завод', 'Имя 1',
       'Поставщик', 'Группа материалов', 'Название группы', 'Краткий текст',
       'Создал', 'Вид докум. закупки.1', 'Обозначение ВидДокум',
       'Дата создания.1', 'Создал.1', 'Группа закупок.1',
       'Название ГрЗакупок.1', 'ЕИ заказа на постав.', 'Единица цены',
       'Цена заказа нетто', 'Стоимость брутто', 'Валюта',
       'Недопоставленное количест

In [86]:
# 1️⃣ Önce df_calisilan_cleaned'deki anahtarları benzersiz hale getiriyoruz
df_calisilan_unique = df_calisilan_cleaned.drop_duplicates(subset='B3-B3POZ')

# 2️⃣ Merge işlemini yapıyoruz (her iki sütunu da ekliyoruz)
df_me5a_cleaned = df_me5a_cleaned.merge(
    df_calisilan_unique[['B3-B3POZ', 'Документ_закупки', 'Зп_Поз']],
    left_on='B3-POZ',
    right_on='B3-B3POZ',
    how='left'
)

# 3️⃣ Dolularını kendi sütunlarına yazıyoruz
df_me5a_cleaned['Заказ на поставку'] = df_me5a_cleaned['Документ_закупки'].combine_first(
    df_me5a_cleaned['Заказ на поставку']
)
df_me5a_cleaned['Позиция ЗкзНаПостав'] = df_me5a_cleaned['Зп_Поз'].combine_first(
    df_me5a_cleaned['Позиция ЗкзНаПостав']
)

# 4️⃣ Geçici sütunları güvenli şekilde sil
cols_to_drop = [col for col in ['Документ_закупки', 'Зп_Поз', 'B3-B3POZ'] if col in df_me5a_cleaned.columns]
df_me5a_cleaned.drop(columns=cols_to_drop, inplace=True)


In [87]:
df_me5a_cleaned.columns

Index(['ГОД', 'Заявка', 'Позиция заявки', 'Материал', 'Индикатор удаления_B3',
       'Затребовал', 'Индик. выдачи', 'Группа деблокир.',
       'Статус обработки заявки', 'Дата заявки', 'Группа закупок',
       'Дата изменения', 'Заказ на поставку', 'Позиция ЗкзНаПостав',
       'Дата заказа', 'Долгосрочный договор', 'ЗатребованКолич',
       'Заказанное к-во', 'Единица измерения', 'RU Наименование',
       'TR Наименование', 'Инвентарный номер', 'Краткий текст',
       'Закуп. организация', 'Создал', 'ПоступлМатериала', 'Поступление счета',
       'Номер проекта', 'Группа материалов', 'Дата поставки',
       'Завод-поставщик', 'Вид документа', 'Статус обработки',
       'Индикатор создания', 'Стратегия деблокир.', 'НДП-материал',
       'Номер потребности', 'Объем дефицита', 'Тип даты поставки',
       'Дата деблокирования', ''Дата заявки', 'zp_zppoz', 'B3-POZ'],
      dtype='object')

In [88]:
df_me5a_cleaned['silme_durumu'] = np.where(
    df_me5a_cleaned['Заказанное к-во'].isna(),  # Eğer boşsa
    'dursun',
    np.where(
        df_me5a_cleaned['Заказанное к-во'] < df_me5a_cleaned['ЗатребованКолич'],
        'dursun',
        'sil'
    )
)

In [89]:
print(df_me5a_cleaned['silme_durumu'].value_counts())

silme_durumu
sil       267498
dursun     90614
Name: count, dtype: int64


In [90]:
df_me5a_cleaned.columns

Index(['ГОД', 'Заявка', 'Позиция заявки', 'Материал', 'Индикатор удаления_B3',
       'Затребовал', 'Индик. выдачи', 'Группа деблокир.',
       'Статус обработки заявки', 'Дата заявки', 'Группа закупок',
       'Дата изменения', 'Заказ на поставку', 'Позиция ЗкзНаПостав',
       'Дата заказа', 'Долгосрочный договор', 'ЗатребованКолич',
       'Заказанное к-во', 'Единица измерения', 'RU Наименование',
       'TR Наименование', 'Инвентарный номер', 'Краткий текст',
       'Закуп. организация', 'Создал', 'ПоступлМатериала', 'Поступление счета',
       'Номер проекта', 'Группа материалов', 'Дата поставки',
       'Завод-поставщик', 'Вид документа', 'Статус обработки',
       'Индикатор создания', 'Стратегия деблокир.', 'НДП-материал',
       'Номер потребности', 'Объем дефицита', 'Тип даты поставки',
       'Дата деблокирования', ''Дата заявки', 'zp_zppoz', 'B3-POZ',
       'silme_durumu'],
      dtype='object')

In [91]:
df_me5a_cleaned = df_me5a_cleaned.drop(
    df_me5a_cleaned[
        df_me5a_cleaned['Заказ на поставку'].notna() &
        (df_me5a_cleaned['silme_durumu'] == 'sil')
    ].index
)

In [92]:
print(df_me5a_cleaned['Заказ на поставку'].value_counts())

Заказ на поставку
3100010337    260
3100004384    119
3100004876    103
3100011876     77
3100011264     54
             ... 
3100000706      1
3100000074      1
3100003102      1
3100000697      1
3100002108      1
Name: count, Length: 1100, dtype: int64


In [93]:
for col in ["Заказ на поставку", "Позиция ЗкзНаПостав"]:
    # Boş değerleri koru, dolu olanları integer yap
    df_me5a_cleaned[col] = df_me5a_cleaned[col].apply(
        lambda x: pd.NA if pd.isna(x) else int(x) if float(x).is_integer() else float(x)
    ).astype("Int64")  # nullable integer dtype

In [94]:
df_me5a_cleaned[['Заявка','Позиция заявки','Заказ на поставку','Заказанное к-во','ЗатребованКолич']].sample(n=5)

,Заявка,Позиция заявки,Заказ на поставку,Заказанное к-во,ЗатребованКолич
29049,2100003227,60,<NA>,0.0,100.0
168091,2200009031,80,<NA>,0.0,2.0
59072,2200013763,50,<NA>,0.0,1.0
207205,9900000387,190,<NA>,0.0,1.0
179315,9900000896,1820,<NA>,0.0,1.0


In [95]:
# 1. Zorunlu temizleme işlemleri (orijinal sütunların üzerine yazar)
df_me5a_cleaned["Заявка"] = df_me5a_cleaned["Заявка"].apply(temizle_ve_cevir)
df_me5a_cleaned["Позиция заявки"] = df_me5a_cleaned["Позиция заявки"].apply(temizle_ve_cevir)

df_zmm067_2022_2025["Заявка"] = df_zmm067_2022_2025["Заявка"].apply(temizle_ve_cevir)
df_zmm067_2022_2025["Позиция заявки"] = df_zmm067_2022_2025["Позиция заявки"].apply(temizle_ve_cevir)

# 2. Anahtar oluştur
df_me5a_cleaned["zayavka_poz"] = df_me5a_cleaned["Заявка"] + "_" + df_me5a_cleaned["Позиция заявки"]
df_zmm067_2022_2025["zayavka_poz"] = df_zmm067_2022_2025["Заявка"] + "_" + df_zmm067_2022_2025["Позиция заявки"]

# 3. Dict oluştur
dict_dop = dict(zip(df_zmm067_2022_2025["zayavka_poz"], df_zmm067_2022_2025["Дополнительное"]))

# 4. map ile yeni sütun ekle
df_me5a_cleaned["Дополнительное"] = df_me5a_cleaned["zayavka_poz"].map(dict_dop)

#yeni oluşturulan sutunlara ihtiyaç kalmadı

df_me5a_cleaned = df_me5a_cleaned.drop(columns=["zayavka_poz"])
df_zmm067_2022_2025 = df_zmm067_2022_2025.drop(columns=["zayavka_poz"])

In [96]:
df_me2n_2022_2025.sample(n=5)

,ГОД,Документ закупки,Позиция,Создал,Группа закупок,Закуп. организация,Индикатор удаления,Дата документа,Наш знак,Ваш код,...,Вид докум. закупки,Код налога,Краткий текст,Дата поставки,ЕИ заказа на постав.,ТипДокумЗакупки,Тип неполноты данных,Группа материалов,Материал_ME2N,Материал_ME5A
266738,2023,3100005559,130,EMILAEVA,115,2991,<NA>,2023-12-08,68/310,190402,...,ZSUP,V0,30UJA.0.KM.LH0004.20/1317.1.EP-1.33,2023-11-05,ШТ,F,Документ полностью сохранен,MZ-2065,4500059363,4500059363
180011,2024,3100008758,1440,IKOVALEV,001,2991,<NA>,2024-08-05,90VFA2024,90VFA2024,...,ZSUP,VB,AKU.0120.10UKC.0.KM.LC0024.SR-20,2024-02-03,ШТ,F,Документ полностью сохранен,MZ-2065,4500279718,4500279718
65171,2025,3100012238,1880,ACHARYEV,100,2992,<NA>,2025-02-28,AKEM331,AKEM331,...,ZSUP,VB,22/224 .0120.10UJA.KAA.TM.TB0204 M38-1,2024-07-06,КГ,F,Документ полностью сохранен,MZ-2065,4500394681,4500394681
22947,2025,3100013901,1260,IBATENKO,002,2992,L,2025-09-05,05.1b-part 1,FT04231034,...,ZSUP,VB,S-5.1a_Pi-1.1_00UYB.SRP.TM.TB0010,2025-10-05,КГ,F,Документ полностью сохранен,MZ-2065,4500502710,4500502710
109468,2024,3100010709,20,ESIKORSKAIA,165,2991,<NA>,2024-10-23,КП-22-274/21,КП-22-274/21,...,ZSUP,V0,AKU.0120.30UJA.0.KM.LC0034 DP1.2,2024-11-21,ШТ,F,Документ полностью сохранен,MZ-2065,4500441512,4500441512


In [97]:
"""
del df_old_me2n
del old_employee
del df_old_me2n_filtered
del df_filtered2
"""

'\ndel df_old_me2n\ndel old_employee\ndel df_old_me2n_filtered\ndel df_filtered2\n'

In [98]:
df_old_me2n=df_me2n_2022_2025.copy()

In [99]:
df_old_me2n.columns

Index(['ГОД', 'Документ закупки', 'Позиция', 'Создал', 'Группа закупок',
       'Закуп. организация', 'Индикатор удаления', 'Дата документа',
       'Наш знак', 'Ваш код', 'Имя поставщика', 'Заявка', 'Позиция заявки',
       'Материал', 'Инвентарный номер', 'Наименование RU', 'Наименование TR',
       'Объем заказа', 'Складская ЕИ', 'еще поставить (количество)',
       'Еще для поставки (стоимость)', 'Уже поставлено (количество)',
       'Для фактурирования (стоим.)', 'Для фактурирования (колич.)',
       'Количество в СЕИ', 'Единица цены', 'СтоимЗаказа нетто', 'Валюта',
       'Цена нетто', 'Вид докум. закупки', 'Код налога', 'Краткий текст',
       'Дата поставки', 'ЕИ заказа на постав.', 'ТипДокумЗакупки',
       'Тип неполноты данных', 'Группа материалов', 'Материал_ME2N',
       'Материал_ME5A'],
      dtype='object')

In [100]:
df_old_me2n['Создал'].unique()

array(['SSTRIAPUKHIN', 'NOZKAN', 'KGAIVUK', 'EKOC', 'MCIMAN', 'FPINAR',
       'ESAVRAN', 'APOTAPOV', 'ZIZZET', 'HALLAKOV', 'MAKIF', 'ACHARYEV',
       'NBEREZOVSKAY', 'ILUKIANCHUK', 'SISIGUZEL', 'DKILIC', 'TEGOROVA',
       'ZDURMAZ', 'VKUZMICHEV', 'MSAHIN', 'IBATENKO', 'FSALIMOV',
       'ETROFIMUSHK', <NA>, 'ISVIRID', 'ESAMSONOVA', 'SOZEVIN', 'NARIKAN',
       'IKALASHNIKOV', 'ZHUSEYNOVA', 'RGAVRIKOVA', 'ARTGALIEV', 'ILAMBIN',
       'NOMAROVA', 'SZHELAMSKII', 'NBULUT', 'ESIKORSKAIA', 'IKOVALEV',
       'MCHEREMNYKH', 'NZYBINSSKAYA', 'SUSLU', 'OSKREBETS', 'MPETROVA',
       'TZADOROZHN', 'SBASTANOV', 'MZHAGIPAROV', 'IABBASOVA', 'MCETIN',
       'KLITVINOVA', 'SYILDIRIM', 'GCEREZCI', 'NKANYSBAY', 'ACOSKUN',
       'RDAVLETBAKOV', 'OLUGOVIK', 'YOZTURK', 'FANNAKULIEVA', 'IMURADIAN',
       'TMISHATKINA', 'GTORAMAN', 'MOZCAN', 'RKAYNAR', 'EUSLU', 'VSOYBIR',
       'OKOKOGLU', 'VPIROG', 'MLEBEDEV', 'ZABDUVAKHAP', 'ESARAL',
       'SMANAFOVA', 'TURMANOV', 'EMILAEVA', 'AMARIA', 'AISAKINA',

In [101]:
"""
old_employee =['TURMANOV','ILAMBIN','IKALASHNIKOV','MOZCAN','ZHUSEYNOVA','ESIKORSKAIA','FANNAKULIEVA',
               'EUSLU','RALIOGLU','GTORAMAN','YOZTURK','GCEREZCI','MMAMAZHANOV','ESARAL','KDUVERLIOGLU',
               'AMARIA','MPETROVA','NKANYSBAY','ACOSKUN','RKAYNAR','EMILAEVA',
               'DPANTEEV','EYILDIZ','VSOYBIR','DSHARKOV','CCANBULAT','VPIROG',
               'TDZHURAEV','TMISHATKINA','MMOROZOV','MMURADOV'
              ]
              """

"\nold_employee =['TURMANOV','ILAMBIN','IKALASHNIKOV','MOZCAN','ZHUSEYNOVA','ESIKORSKAIA','FANNAKULIEVA',\n               'EUSLU','RALIOGLU','GTORAMAN','YOZTURK','GCEREZCI','MMAMAZHANOV','ESARAL','KDUVERLIOGLU',\n               'AMARIA','MPETROVA','NKANYSBAY','ACOSKUN','RKAYNAR','EMILAEVA',\n               'DPANTEEV','EYILDIZ','VSOYBIR','DSHARKOV','CCANBULAT','VPIROG',\n               'TDZHURAEV','TMISHATKINA','MMOROZOV','MMURADOV'\n              ]\n              "

In [102]:
old_employee =['NBEREZOVSKAY', 'NOZKAN', 'DKILIC', 'ESAVRAN', 'HALLAKOV',
       'ACHARYEV', 'MCIMAN', 'APOTAPOV', 'TEGOROVA', 'EKOC', 'IBATENKO',
       'ZIZZET', 'SISIGUZEL', 'FPINAR', 'KGAIVUK', 'MAKIF', 'MSAHIN',
       'FSALIMOV', 'SSTRIAPUKHIN', 'VKUZMICHEV', 'ETROFIMUSHK',
       'ILUKIANCHUK', 'ZDURMAZ', 'ISVIRID', 'ESAMSONOVA', 'SOZEVIN',
       'NARIKAN', 'IKALASHNIKOV', 'ZHUSEYNOVA', 'RGAVRIKOVA', 'ARTGALIEV',
       'ILAMBIN', 'NOMAROVA', 'SZHELAMSKII', 'NBULUT', 'ESIKORSKAIA',
       'IKOVALEV', 'MCHEREMNYKH', 'NZYBINSSKAYA', 'SUSLU', 'OSKREBETS',
       'TZADOROZHN', 'MPETROVA', 'SBASTANOV', 'MZHAGIPAROV', 'IABBASOVA',
       'MCETIN', 'KLITVINOVA', 'SYILDIRIM', 'GCEREZCI', 'NKANYSBAY',
       'ACOSKUN', 'RDAVLETBAKOV', 'OLUGOVIK', 'YOZTURK', 'FANNAKULIEVA',
       'TMISHATKINA', 'IMURADIAN', 'GTORAMAN', 'MOZCAN', 'RKAYNAR',
       'EUSLU', 'VSOYBIR', 'OKOKOGLU', 'VPIROG', 'MLEBEDEV',
       'ZABDUVAKHAP', 'ESARAL', 'SMANAFOVA', 'TURMANOV', 'EMILAEVA',
       'AMARIA', 'AISAKINA', 'ACHERKASOV', 'CCANBULAT', 'EYILDIZ',
       'MANDREEV', 'EPULATOV', 'MMOROZOV', 'MMAMAZHANOV', 'KDUVERLIOGLU',
       'TDZHURAEV', 'NTUAEVA', 'TAPARINA', 'IKUCUKKAVRUK', 'DSHARKOV',
       'AEIGOSHEV', 'IZEMLIAKOVA', 'MMURADOV', 'MBURMAKOVA', 'SBAKHITOV',
       'EGASIMOV', 'SURMANTSEVA', 'AEROKHOV', 'NHASANLI', 'MVOLKOVA',
       'RMAMMADOV', 'LUFITSEVA', 'IISMAILOV', 'EKHUDIAKOVA',
       'FMUSTAFAZADE', 'RALIOGLU', 'DPANTEEV', 'ZBONDAREVA'
              ]

In [103]:
df_old_me2n = df_old_me2n[df_old_me2n['Создал'].isin(old_employee)]

In [104]:
df_old_me2n.columns

Index(['ГОД', 'Документ закупки', 'Позиция', 'Создал', 'Группа закупок',
       'Закуп. организация', 'Индикатор удаления', 'Дата документа',
       'Наш знак', 'Ваш код', 'Имя поставщика', 'Заявка', 'Позиция заявки',
       'Материал', 'Инвентарный номер', 'Наименование RU', 'Наименование TR',
       'Объем заказа', 'Складская ЕИ', 'еще поставить (количество)',
       'Еще для поставки (стоимость)', 'Уже поставлено (количество)',
       'Для фактурирования (стоим.)', 'Для фактурирования (колич.)',
       'Количество в СЕИ', 'Единица цены', 'СтоимЗаказа нетто', 'Валюта',
       'Цена нетто', 'Вид докум. закупки', 'Код налога', 'Краткий текст',
       'Дата поставки', 'ЕИ заказа на постав.', 'ТипДокумЗакупки',
       'Тип неполноты данных', 'Группа материалов', 'Материал_ME2N',
       'Материал_ME5A'],
      dtype='object')

In [105]:
df_old_me2n['Индикатор удаления'].nunique()

2

In [106]:
df_old_me2n=df_old_me2n[(df_old_me2n['Еще для поставки (стоимость)']>0) & (df_old_me2n['Индикатор удаления'].isna())]

In [107]:
df_old_me2n['Индикатор удаления'].nunique()

0

In [108]:
# Gruplama için kullanılacak sütunlar
group_cols = ['Документ закупки']
# Aggregation (toplama/birleştirme) fonksiyonları
agg_funcs = {

    'Создал':'first',
    'Группа закупок':'first' ,
    'Закуп. организация':'first',
    'Дата документа':'first',
    'Наш знак':'first'   ,
    'Ваш код':'first',
    'Имя поставщика':'first',
    'СтоимЗаказа нетто':'sum',
    'Еще для поставки (стоимость)':'sum',
    'Валюта':'first'
}

# Gruplama ve aggregation işlemini gerçekleştiriyoruz
df_old_me2n_filtered = df_old_me2n.groupby(group_cols, as_index=False).agg(agg_funcs)

# The filtering for 'Индикатор удаления' was already performed on df_old_me2n
df_old_me2n_filtered=df_old_me2n_filtered[df_old_me2n_filtered['Еще для поставки (стоимость)']>0]

In [109]:
df_old_me2n_filtered = df_old_me2n_filtered.drop(columns=['Имя поставщика'])

In [110]:
df_old_me2n_filtered.columns

Index(['Документ закупки', 'Создал', 'Группа закупок', 'Закуп. организация',
       'Дата документа', 'Наш знак', 'Ваш код', 'СтоимЗаказа нетто',
       'Еще для поставки (стоимость)', 'Валюта'],
      dtype='object')

In [111]:
df_calisilan['Поставщик']

,Поставщик
0,<NA>
1,<NA>
2,<NA>
3,<NA>
4,<NA>
...,...
767882,1000019187
767883,1000019187
767884,1000019187
767885,1000001070


In [112]:
df_calisilan.columns

Index(['ГОД', 'zp_zppoz', 'B3-B3POZ', 'Документ_закупки', 'Зп_Поз',
       'Индикатор удаления', 'неполн.', 'Материал', 'Заявка', 'Позиция заявки',
       'RU Наименование', 'TR Наименование', 'Инвентарный номер в SAS',
       'Количество ЗП', 'Минимальная цена ЗП', 'Минимальная цена ЗП.1',
       'Минимальная цена ЗП.2', 'Единица ЗП', 'Цена нетто ЗП',
       'Итого цена нетто', 'Валюта ЗП', 'Вид докум. закупки', 'Дата создания',
       'Дата поставки', 'Дата доставки, одобренная Поставщиком',
       'Группа закупок', 'Название ГрЗакупок', 'БЕ', 'Название фирмы',
       'Закуп. организация', 'Название ЗакупОрг', 'Завод', 'Имя 1',
       'Поставщик', 'Группа материалов', 'Название группы', 'Краткий текст',
       'Создал', 'Вид докум. закупки.1', 'Обозначение ВидДокум',
       'Дата создания.1', 'Создал.1', 'Группа закупок.1',
       'Название ГрЗакупок.1', 'ЕИ заказа на постав.', 'Единица цены',
       'Цена заказа нетто', 'Стоимость брутто', 'Валюта',
       'Недопоставленное количест

In [113]:
# 1. Anahtar sütunları tuple haline getirelim
df_calisilan["key"] = df_calisilan[["Документ_закупки"]].apply(tuple, axis=1)
df_old_me2n_filtered["key"] = df_old_me2n_filtered[["Документ закупки"]].apply(tuple, axis=1)

# 2. Aktarılacak sütunlar ve hedef adları
map_dict_B3_status = {
    "имя_поставщика":"имя_поставщика",
    "Поставщик":"Номер Поставщика SAP"
}

# 3. Her sütun için dict map uygulama
for source_col, target_col in map_dict_B3_status.items():
    temp_dict = dict(zip(df_calisilan["key"], df_calisilan[source_col]))
    df_old_me2n_filtered[target_col] = df_old_me2n_filtered["key"].map(temp_dict)

# 4. Geçici key sütunlarını kaldır
df_old_me2n_filtered.drop(columns=["key"], inplace=True)
df_calisilan.drop(columns=["key"], inplace=True)

In [114]:
df_old_me2n_filtered.sample(n=5)

,Документ закупки,Создал,Группа закупок,Закуп. организация,Дата документа,Наш знак,Ваш код,СтоимЗаказа нетто,Еще для поставки (стоимость),Валюта,имя_поставщика,Номер Поставщика SAP
40,3100000850,ZDURMAZ,106,2991,2023-02-17,AKC 157,AKC 157,2527.66,2527.66,USD,ADA END.ÜRN. İNŞ.NAK.SAN.İC VE DIŞ,1000001811
647,3100004688,MZHAGIPAROV,002,2992,2023-11-03,POAKЕM155,AKЕM155,71.24,71.24,USD,İNCE İŞ METAL PROJE İNŞ. MAKİNA,1000001420
2208,3100011951,VKUZMICHEV,001,2991,2025-02-06,90VFA2024,90VFA2024,6727.12,6501.06,USD,IC IÇTAS ENDÜSTRIYEL BORU LOJISTIK,2020
391,3100003545,MAKIF,001,2991,2023-08-23,AK-NP-1826/2,AK-NP-1826/2,32040.00,0.81,USD,DK NET ENERJİ SANAYİ VE TİCARET A.Ş,1000001142
2826,3100013784,NBEREZOVSKAY,103,2991,2025-08-01,HCN230160,HCN230160,1876.90,1876.90,EUR,HİLTİ İNŞAAT MALZEMELERİ TİC. A.Ş.,1000001086


In [115]:
#her bir kuratörü tek tek kaydetme
# 1. Gerekli sütun sırasını tanımla
columns_order = [
    'Создал','Группа закупок','Документ закупки','Закуп. организация','Дата документа','Наш знак','Ваш код','имя_поставщика','Номер Поставщика SAP','СтоимЗаказа нетто','Еще для поставки (стоимость)','Валюта'
]

# 2. Benzersiz Заявитель değerlerini al
employees = df_old_me2n_filtered['Создал'].dropna().unique()

# 3. Çıktıları kaydetmek için klasör oluştur (varsa geç)
os.makedirs("not_delivered", exist_ok=True)

# 4. Her Заявитель için döngü
for z_name in employees:
    # Dosya adı için geçersiz karakterleri temizle
    safe_name = "".join(c if c.isalnum() or c in "._-" else "_" for c in z_name)

    # Filtreleme ve sütun sıralama
    df_filtered2 = df_old_me2n_filtered[df_old_me2n_filtered['Создал'] == z_name].copy()

    # Sütun kontrolü: eksik olanları atla (varsa uyarı verir)
    missing_cols = [col for col in columns_order if col not in df_filtered2.columns]
    if missing_cols:
        print(f"{z_name} için eksik sütun(lar): {missing_cols}")
        continue

    df_filtered2 = df_filtered2[columns_order]

    # Excel olarak kaydet
    output_path = f"not_delivered/me2n_notdelivered_{safe_name}.xlsx"
    df_filtered2.to_excel(output_path, index=False)

print("✅ Tüm dosyalar 'not_delivered/' klasörüne kaydedildi.")

✅ Tüm dosyalar 'not_delivered/' klasörüne kaydedildi.


In [116]:
zip_filename = "not_delivered_me2n_all.zip"

with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk("not_delivered"):
        for file in files:
            if file.endswith(".xlsx"):
                file_path = os.path.join(root, file)
                zipf.write(file_path)

In [117]:
from google.colab import files
files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [118]:
#sadece geliştirme aşamasında kullanılacak !
"""
del zmb51_dummy
del df_B3_status
del zmm059_dummy
del me2n_dummy
del zpp001_dummy

"""

'\ndel zmb51_dummy\ndel df_B3_status\ndel zmm059_dummy\ndel me2n_dummy\ndel zpp001_dummy\n\n'

In [119]:
df_B3_status=df_me5a_2022_2025.copy()
zmb51_dummy=birlesik_df_zmb51.copy()
zmm059_dummy=df_calisilan.copy()
me2n_dummy=df_me2n_2022_2025.copy()
zpp001_dummy=zpp001.copy()

In [120]:
zmb51_dummy.columns

Index(['ГОД', 'Вид движения', 'Партия', 'Материал', 'Заказ на поставку',
       'Позиция', 'Наш знак', 'Ваш код', 'Внутренний заказ', 'Позиция заявки',
       'Склад', 'Дата ввода', 'Время ввода', 'Дата документа', 'Дата проводки',
       'Документ материала', 'Поз. док. материала', 'Признак 2', 'Ссылка',
       'Краткий текст материала', 'Кол-во в ЕИ ввода', 'Объем заказа',
       'ЕИ ввода', 'Сумма во ВВ', 'Количество', 'Поставщик', 'Имя поставщика',
       'Накладная', 'Текст заголовка документа', 'Заявитель',
       'Инвентарный номер', 'Имя пользователя', 'БЕ', 'Базовая ЕИ',
       'ГодДокумМатериала', 'Валюта', 'Завод', 'Имя 1', 'Основное средство',
       'ЕИ цены заказа', 'ЕИ заказа на постав.'],
      dtype='object')

In [121]:
display(zmb51_dummy[(zmb51_dummy["Заказ на поставку"]==3100005467) & (zmb51_dummy["Позиция"]==80)][["Кол-во в ЕИ ввода", "Сумма во ВВ", "Наш знак"]])

,Кол-во в ЕИ ввода,Сумма во ВВ,Наш знак
285211,1.0,14511.18,AK-C-389


In [122]:
#zmb51_dummy = duzenle_sayi_sutunu(zmb51_dummy, "Кол-во в ЕИ ввода")

In [123]:
zmb51_dummy['Признак 2&кол-во&ЕИ']=zmb51_dummy['Признак 2'].astype(str)+' '+zmb51_dummy['Кол-во в ЕИ ввода'].astype(str)+' '+zmb51_dummy['ЕИ ввода'].astype(str)
zmb51_dummy['Ссылка&кол-во&ЕИ']=zmb51_dummy['Ссылка'].astype(str)+' '+zmb51_dummy['Кол-во в ЕИ ввода'].astype(str)+' '+zmb51_dummy['ЕИ ввода'].astype(str)
zmb51_dummy["Кол-во в ЕИ ввода"] = pd.to_numeric(zmb51_dummy["Кол-во в ЕИ ввода"], errors="coerce").astype("float64")


In [124]:
zmb51_dummy.columns

Index(['ГОД', 'Вид движения', 'Партия', 'Материал', 'Заказ на поставку',
       'Позиция', 'Наш знак', 'Ваш код', 'Внутренний заказ', 'Позиция заявки',
       'Склад', 'Дата ввода', 'Время ввода', 'Дата документа', 'Дата проводки',
       'Документ материала', 'Поз. док. материала', 'Признак 2', 'Ссылка',
       'Краткий текст материала', 'Кол-во в ЕИ ввода', 'Объем заказа',
       'ЕИ ввода', 'Сумма во ВВ', 'Количество', 'Поставщик', 'Имя поставщика',
       'Накладная', 'Текст заголовка документа', 'Заявитель',
       'Инвентарный номер', 'Имя пользователя', 'БЕ', 'Базовая ЕИ',
       'ГодДокумМатериала', 'Валюта', 'Завод', 'Имя 1', 'Основное средство',
       'ЕИ цены заказа', 'ЕИ заказа на постав.', 'Признак 2&кол-во&ЕИ',
       'Ссылка&кол-во&ЕИ'],
      dtype='object')

In [125]:
# Gruplama için kullanılacak sütunlar
group_cols = ['Заказ на поставку', 'Позиция']
# Aggregation (toplama/birleştirme) fonksiyonları
agg_funcs = {
    'Кол-во в ЕИ ввода': 'sum', # 'Кол-во в ЕИ ввода' sütunundaki değerleri topluyoruz
    'Дата проводки': lambda x: ';'.join(x.dropna().astype(str)),
    # 'Дата проводки' sütununu 'YYYY-MM-DD' formatında stringe çevirip,
    # benzersiz değerleri alıp ';' ile birleştiriyoruz. NaN değerler dışarıda bırakılır.
    'Признак 2&кол-во&ЕИ': lambda x: ';'.join(x.dropna().astype(str)),
    # 'накладная&кол-во&ЕИ 1' sütunundaki benzersiz string değerleri ';' ile birleştiriyoruz. NaN değerler dışarıda bırakılır.
    'Ссылка&кол-во&ЕИ': lambda x: ';'.join(x.dropna().astype(str))
    # 'накладная&кол-во&ЕИ 2' sütunundaki benzersiz string değerleri ';' ile birleştiriyoruz. NaN değerler dışarıda bırakılır.
}

# Gruplama ve aggregation işlemini gerçekleştiriyoruz
result_df = zmb51_dummy.groupby(group_cols, as_index=False).agg(agg_funcs)
del zmb51_dummy
zmb51_dummy=result_df.copy()
del result_df

In [126]:
zmb51_dummy["Заказ на поставку"] = zmb51_dummy["Заказ на поставку"].apply(temizle_ve_cevir)
zmb51_dummy["Позиция"] = zmb51_dummy["Позиция"].apply(temizle_ve_cevir)
me2n_dummy["Документ закупки"] = me2n_dummy["Документ закупки"].apply(temizle_ve_cevir)
me2n_dummy["Позиция"] = me2n_dummy["Позиция"].apply(temizle_ve_cevir)

In [127]:
me2n_dummy.columns

Index(['ГОД', 'Документ закупки', 'Позиция', 'Создал', 'Группа закупок',
       'Закуп. организация', 'Индикатор удаления', 'Дата документа',
       'Наш знак', 'Ваш код', 'Имя поставщика', 'Заявка', 'Позиция заявки',
       'Материал', 'Инвентарный номер', 'Наименование RU', 'Наименование TR',
       'Объем заказа', 'Складская ЕИ', 'еще поставить (количество)',
       'Еще для поставки (стоимость)', 'Уже поставлено (количество)',
       'Для фактурирования (стоим.)', 'Для фактурирования (колич.)',
       'Количество в СЕИ', 'Единица цены', 'СтоимЗаказа нетто', 'Валюта',
       'Цена нетто', 'Вид докум. закупки', 'Код налога', 'Краткий текст',
       'Дата поставки', 'ЕИ заказа на постав.', 'ТипДокумЗакупки',
       'Тип неполноты данных', 'Группа материалов', 'Материал_ME2N',
       'Материал_ME5A'],
      dtype='object')

In [128]:
# 1. Anahtar sütunları tuple haline getirelim
me2n_dummy["key"] = me2n_dummy[["Документ закупки", "Позиция"]].apply(tuple, axis=1)
zmb51_dummy["key"] = zmb51_dummy[["Заказ на поставку", "Позиция"]].apply(tuple, axis=1)

# 2. Aktarılacak sütunlar ve hedef adları
map_dict_B3_status = {
    "Кол-во в ЕИ ввода": "Кол-во в ЕИ ввода",
    "Дата проводки":"Дата проводки",
    "Признак 2&кол-во&ЕИ":"Признак 2&кол-во&ЕИ",
    "Ссылка&кол-во&ЕИ":"Ссылка&кол-во&ЕИ",


}

# 3. Her sütun için dict map uygulama
for source_col, target_col in map_dict_B3_status.items():
    temp_dict = dict(zip(zmb51_dummy["key"], zmb51_dummy[source_col]))
    me2n_dummy[target_col] = me2n_dummy["key"].map(temp_dict)

# 4. Geçici key sütunlarını kaldır
zmb51_dummy.drop(columns=["key"], inplace=True)
me2n_dummy.drop(columns=["key"], inplace=True)

In [129]:
zmm059_dummy["Документ_закупки"] = zmm059_dummy["Документ_закупки"].apply(temizle_ve_cevir)
zmm059_dummy["Зп_Поз"] = zmm059_dummy["Зп_Поз"].apply(temizle_ve_cevir)
me2n_dummy["Документ закупки"] = me2n_dummy["Документ закупки"].apply(temizle_ve_cevir)
me2n_dummy["Позиция"] = me2n_dummy["Позиция"].apply(temizle_ve_cevir)

In [130]:
# 1. Anahtar sütunları tuple haline getirelim
me2n_dummy["key"] = me2n_dummy[["Документ закупки", "Позиция"]].apply(tuple, axis=1)
zmm059_dummy["key"] = zmm059_dummy[["Документ_закупки", "Зп_Поз"]].apply(tuple, axis=1)

# 2. Aktarılacak sütunlar ve hedef adları
map_dict_56 = {
    'Группа закупок':'Группа закупок_me2n',
    'Закуп. организация':'Закуп. организация_me2n',
    'Наш знак':'Наш знак_me2n',
    'Ваш код':'Ваш код_me2n',
    'Объем заказа':'Объем заказа_me2n',
    'Материал':'Материал_me2n',
    "Складская ЕИ":"Складская ЕИ_me2n",
    "ЕИ заказа на постав.":"ЕИ заказа на постав._me2n",
    "Кол-во в ЕИ ввода": "Кол-во в ЕИ ввода",
    "Дата проводки":"Дата проводки_me2n",
    "Признак 2&кол-во&ЕИ":"Признак 2&кол-во&ЕИ",
    "Ссылка&кол-во&ЕИ":"Ссылка&кол-во&ЕИ",



}

# 3. Her sütun için dict map uygulama
for source_col, target_col in map_dict_56.items():
    temp_dict = dict(zip(me2n_dummy["key"], me2n_dummy[source_col]))
    zmm059_dummy[target_col] = zmm059_dummy["key"].map(temp_dict)

# 4. Geçici key sütunlarını kaldır
zmm059_dummy.drop(columns=["key"], inplace=True)
me2n_dummy.drop(columns=["key"], inplace=True)

In [131]:
zmm059_dummy["Закуп. организация_me2n"]=zmm059_dummy["Закуп. организация_me2n"].apply(temizle_ve_cevir)

In [132]:
zmm059_dummy.loc[zmm059_dummy["Индикатор удаления"].notna(), "Объем заказа_me2n"] = pd.NA

In [133]:
zmm059_dummy.loc[zmm059_dummy["Индикатор удаления"].notna(), "Индикатор удаления"] = zmm059_dummy["Индикатор удаления"]+"_"+zmm059_dummy["Документ_закупки"]+"-"+zmm059_dummy["Зп_Поз"]

In [134]:
zmm059_dummy[["Индикатор удаления","Объем заказа_me2n"]].sample(n=5 , random_state=42)

,Индикатор удаления,Объем заказа_me2n
498762,<NA>,NaN
733087,<NA>,NaN
254752,<NA>,1.0
741534,<NA>,NaN
641160,L_3100003395-130,NaN


In [135]:
zmm059_dummy.columns

Index(['ГОД', 'zp_zppoz', 'B3-B3POZ', 'Документ_закупки', 'Зп_Поз',
       'Индикатор удаления', 'неполн.', 'Материал', 'Заявка', 'Позиция заявки',
       'RU Наименование', 'TR Наименование', 'Инвентарный номер в SAS',
       'Количество ЗП', 'Минимальная цена ЗП', 'Минимальная цена ЗП.1',
       'Минимальная цена ЗП.2', 'Единица ЗП', 'Цена нетто ЗП',
       'Итого цена нетто', 'Валюта ЗП', 'Вид докум. закупки', 'Дата создания',
       'Дата поставки', 'Дата доставки, одобренная Поставщиком',
       'Группа закупок', 'Название ГрЗакупок', 'БЕ', 'Название фирмы',
       'Закуп. организация', 'Название ЗакупОрг', 'Завод', 'Имя 1',
       'Поставщик', 'Группа материалов', 'Название группы', 'Краткий текст',
       'Создал', 'Вид докум. закупки.1', 'Обозначение ВидДокум',
       'Дата создания.1', 'Создал.1', 'Группа закупок.1',
       'Название ГрЗакупок.1', 'ЕИ заказа на постав.', 'Единица цены',
       'Цена заказа нетто', 'Стоимость брутто', 'Валюта',
       'Недопоставленное количест

In [136]:
# Gruplama için kullanılacak sütunlar
group_cols = ['Заявка', 'Позиция заявки']
# Aggregation (toplama/birleştirme) fonksiyonları
agg_funcs = {

    'Документ_закупки': lambda x: ';'.join(x.dropna().astype(str)),
    'Зп_Поз': lambda x : ';'.join(x.dropna().astype(str)),
    'Индикатор удаления': lambda x : ';'.join(x.dropna().astype(str)),
    'имя_поставщика': lambda x : ';'.join(x.dropna().astype(str)),
    'Группа закупок_me2n': lambda x : ';'.join(x.dropna().astype(str)),
    'Закуп. организация_me2n': lambda x : ';'.join(x.dropna().astype(str)),
    'Наш знак_me2n': lambda x : ';'.join(x.dropna().astype(str)),
    'Ваш код_me2n': lambda x : ';'.join(x.dropna().astype(str)),
    'Объем заказа_me2n': 'sum',
    'Материал_me2n': lambda x : ';'.join(x.dropna().astype(str)),
    'Складская ЕИ_me2n': lambda x : ';'.join(x.dropna().astype(str)),
    'ЕИ заказа на постав._me2n': lambda x : ';'.join(x.dropna().astype(str)),
    'Кол-во в ЕИ ввода': 'sum',
    'Признак 2&кол-во&ЕИ': lambda x : ';'.join(x.dropna().astype(str)),
    'Ссылка&кол-во&ЕИ': lambda x : ';'.join(x.dropna().astype(str)),
    'Дата проводки_me2n': lambda x : ';'.join(x.dropna().astype(str))
}

# Gruplama ve aggregation işlemini gerçekleştiriyoruz
result_df = zmm059_dummy.groupby(group_cols, as_index=False).agg(agg_funcs)
del zmm059_dummy
zmm059_dummy=result_df.copy()
del result_df

In [137]:
zmm059_dummy["Заявка"] = zmm059_dummy["Заявка"].apply(temizle_ve_cevir)
zmm059_dummy["Позиция заявки"] = zmm059_dummy["Позиция заявки"].apply(temizle_ve_cevir)
df_B3_status["Заявка"] = df_B3_status["Заявка"].apply(temizle_ve_cevir)
df_B3_status["Позиция заявки"] = df_B3_status["Позиция заявки"].apply(temizle_ve_cevir)

In [138]:
df_B3_status=df_B3_status.drop(columns=['Объем заказа','Имя поставщика',
                                        'Наш знак', 'Цена нетто', 'Валюта', 'ЕИ заказа на постав.',
                                        'Единица цены', 'Заказ на поставку', 'Позиция ЗкзНаПостав',
                                        'Индикатор удаления_зп','Индик. выдачи','Статус обработки заявки',
                                        'Дата изменения', 'Дата заказа','Долгосрочный договор','Закуп. организация','Создал',
                                        'ПоступлМатериала', 'Поступление счета', 'Номер проекта',
                                        'Дата поставки', 'Завод-поставщик','Вид документа', 'Статус обработки', 'Индикатор создания',
                                        'Стратегия деблокир.', 'НДП-материал', 'Номер потребности',
                                        'Объем дефицита', 'Тип даты поставки', 'Дата деблокирования','B3-B3POZ'

                                        ])

In [139]:
# 1. Anahtar sütunları tuple haline getirelim
df_B3_status["key"] = df_B3_status[["Заявка", "Позиция заявки"]].apply(tuple, axis=1)
zmm059_dummy["key"] = zmm059_dummy[["Заявка", "Позиция заявки"]].apply(tuple, axis=1)

# 2. Aktarılacak sütunlar ve hedef adları
map_dict_B3_status = {
    "Документ_закупки":"Документ_закупки",
    "Зп_Поз":"Зп_Поз",
    "Индикатор удаления":"Индикатор удаления",
    "имя_поставщика":"имя_поставщика",
    "Группа закупок_me2n":"Группа закупок_me2n",
    "Закуп. организация_me2n":"Закуп. организация_me2n",
    "Наш знак_me2n":"Наш знак_me2n",
    "Ваш код_me2n":"Ваш код_me2n",
    "Объем заказа_me2n":"Объем заказа_me2n",
    "Материал_me2n":"Материал_me2n",
    "Складская ЕИ_me2n":"Складская ЕИ_me2n",
    "ЕИ заказа на постав._me2n":"ЕИ заказа на постав._me2n",
    "Кол-во в ЕИ ввода":"Кол-во в ЕИ ввода",
    "Дата проводки_me2n":"Дата проводки",
    "Признак 2&кол-во&ЕИ":"Признак 2&кол-во&ЕИ",
    "Ссылка&кол-во&ЕИ":"Ссылка&кол-во&ЕИ",

}

# 3. Her sütun için dict map uygulama
for source_col, target_col in map_dict_B3_status.items():
    temp_dict = dict(zip(zmm059_dummy["key"], zmm059_dummy[source_col]))
    df_B3_status[target_col] = df_B3_status["key"].map(temp_dict)

# 4. Geçici key sütunlarını kaldır
zmm059_dummy.drop(columns=["key"], inplace=True)
df_B3_status.drop(columns=["key"], inplace=True)

In [140]:
df_B3_status.columns

Index(['ГОД', 'Заявка', 'Позиция заявки', 'Материал', 'Индикатор удаления_B3',
       'ЗатребованКолич', 'Затребовал', 'Группа деблокир.', 'Дата заявки',
       'Группа закупок', 'Заказанное к-во', 'Единица измерения',
       'RU Наименование', 'TR Наименование', 'Дополнительное',
       'Инвентарный номер', 'Краткий текст', 'еще поставить (количество)',
       'Еще для поставки (стоимость)', 'Для фактурирования (колич.)',
       'Для фактурирования (стоим.)', 'ЗП_Создал', 'Группа материалов',
       'Дата_заявки', 'Материал_ME5A', 'Документ_закупки', 'Зп_Поз',
       'Индикатор удаления', 'имя_поставщика', 'Группа закупок_me2n',
       'Закуп. организация_me2n', 'Наш знак_me2n', 'Ваш код_me2n',
       'Объем заказа_me2n', 'Материал_me2n', 'Складская ЕИ_me2n',
       'ЕИ заказа на постав._me2n', 'Кол-во в ЕИ ввода', 'Дата проводки',
       'Признак 2&кол-во&ЕИ', 'Ссылка&кол-во&ЕИ'],
      dtype='object')

In [141]:
df_B3_status["Закуп. организация_me2n"].sample(n=5)

,Закуп. организация_me2n
13312,2991
81748,2992
204992,2991
301888,2991
120346,2991


In [142]:
df_B3_status.columns

Index(['ГОД', 'Заявка', 'Позиция заявки', 'Материал', 'Индикатор удаления_B3',
       'ЗатребованКолич', 'Затребовал', 'Группа деблокир.', 'Дата заявки',
       'Группа закупок', 'Заказанное к-во', 'Единица измерения',
       'RU Наименование', 'TR Наименование', 'Дополнительное',
       'Инвентарный номер', 'Краткий текст', 'еще поставить (количество)',
       'Еще для поставки (стоимость)', 'Для фактурирования (колич.)',
       'Для фактурирования (стоим.)', 'ЗП_Создал', 'Группа материалов',
       'Дата_заявки', 'Материал_ME5A', 'Документ_закупки', 'Зп_Поз',
       'Индикатор удаления', 'имя_поставщика', 'Группа закупок_me2n',
       'Закуп. организация_me2n', 'Наш знак_me2n', 'Ваш код_me2n',
       'Объем заказа_me2n', 'Материал_me2n', 'Складская ЕИ_me2n',
       'ЕИ заказа на постав._me2n', 'Кол-во в ЕИ ввода', 'Дата проводки',
       'Признак 2&кол-во&ЕИ', 'Ссылка&кол-во&ЕИ'],
      dtype='object')

In [143]:
display(df_B3_status[(df_B3_status["Заявка"]=="2200006369") & (df_B3_status["Позиция заявки"]=="5320")][["Кол-во в ЕИ ввода"]])

,Кол-во в ЕИ ввода
293255,4.0


In [144]:
df_B3_status[["Документ_закупки","Зп_Поз"]].sample(n=5)

,Документ_закупки,Зп_Поз
124201,3100011070,740
187679,nan,nan
197334,3100010337,21180
149693,3100008761,100
101408,nan,nan


In [145]:
zpp001_dummy['Purpose Code'].sample(n=5)

,Purpose Code
5359,"кабель, 70% аванс"
6331,Материалы для нужд TSM
3879,смесь сухая бетонная
1574,Оплата за поставку ПМК /ММР
8218,ABRASİVE MATERİYAL ALIMI


In [146]:
zpp001_dummy.columns

Index(['﻿Валюта оплаты', 'Номер заявки на платеж', 'Тип транзакции',
       'Тип документа', 'Тип платежа', 'Статья ДДС', 'Наименование',
       'Purpose Code', 'Протокол', 'Номер договора', 'Инвойс', 'Кредитор',
       'Название кредитора', 'Оплачиваемая сумма', 'Валюта', 'Автор изменения',
       'Дата изменения', 'Создал', 'Дата создания', 'Unnamed: 19',
       'Статус согласующего', 'Оплаченная Сумма', 'Unnamed: 22', 'Дебитор',
       'Название заказчика', 'Unnamed: 25', 'Документ закупки', 'Unnamed: 27',
       'Unnamed: 28', 'Статус', 'Время', 'Время изменения', 'Дата платежа'],
      dtype='object')

In [147]:
zpp001[["Purpose Code","Номер заявки на платеж","Протокол","Номер договора","Инвойс"]].sample(n=5 ,random_state=38)

,Purpose Code,Номер заявки на платеж,Протокол,Номер договора,Инвойс
6645,переключатеоь кулачковый,100000005651,AK-EM-511,AK-EM-511,PROFORMA-FATURA
7635,Расходные материалы,100000003311,AK-NP-1903,PO AK-NP-1903,100037372
3843,поставка технических газов,100000012865,AK-NP-EM-754,TSM-07-22-859,100065109
2507,Подопорные металлоконструкции,100000017034,SP13,TSM-07-22-891 SP13,100011111
8039,Переносная радиостанция,100000002150,AK-NP-2282,AK-NP-2282,PROFORMA-FATURA


In [148]:
"""
import re
import pandas as pd

def extract_text_outside_quotes(zp_string):
    if pd.isna(zp_string) or not isinstance(zp_string, str):
        return None

    # Use regex to remove the quoted part (including the quotes)
    # The '.*?' makes the match non-greedy
    text_outside_quotes = re.sub(r'"(.*?)"', '', zp_string)

    # Trim any leading/trailing whitespace from the remaining text
    trimmed_text = text_outside_quotes.strip()

    # Return None if the result is an empty string, otherwise return the trimmed text
    return trimmed_text if trimmed_text else None

def format_zp_numbers(zp_string):
    if pd.isna(zp_string) or not isinstance(zp_string, str) or not zp_string.strip():
        return None

    # Try to extract content within double quotes first, for existing formats
    match_quoted = re.search(r'"([^"]*)"', zp_string)
    if match_quoted:
        # If quotes found, process only the content within them
        content_to_process = match_quoted.group(1)
    else:
        # If no quotes found, clean the whole string and process it
        content_to_process = zp_string.strip()

    # Find all '31' followed by digits, optionally with a hyphen, in the content_to_process
    # This will catch patterns like '31-14677', '3114677', '31-1234'
    # It ensures that only sequences starting with '31' and followed by digits are considered.
    zp_number_patterns = re.findall(r'31-?\d+', content_to_process)

    formatted_numbers = []
    for num_str in zp_number_patterns:
        # Remove any hyphens from the extracted pattern (e.g., '31-14677' -> '3114677')
        clean_num = num_str.replace('-', '')

        if clean_num.startswith('31') and len(clean_num) > 2:
            numeric_part = clean_num[2:] # Get the part after '31'
            if numeric_part.isdigit():
                # Pad with leading zeros to make the part after '31' 8 digits long
                padded_part = numeric_part.zfill(8);
                formatted_numbers.append('31' + padded_part)
            # If numeric_part is not digits, we skip it as it's not a valid ZP number part
        # If it doesn't start with '31' or is too short, we skip it
        # This function focuses specifically on '31' numbers.

    return ';'.join(formatted_numbers) if formatted_numbers else None
    """

<>:35: SyntaxWarning: invalid escape sequence '\d'
<>:35: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-3089327494.py:35: SyntaxWarning: invalid escape sequence '\d'
  zp_number_patterns = re.findall(r'31-?\d+', content_to_process)


'\nimport re\nimport pandas as pd\n\ndef extract_text_outside_quotes(zp_string):\n    if pd.isna(zp_string) or not isinstance(zp_string, str):\n        return None\n\n    # Use regex to remove the quoted part (including the quotes)\n    # The \'.*?\' makes the match non-greedy\n    text_outside_quotes = re.sub(r\'"(.*?)"\', \'\', zp_string)\n\n    # Trim any leading/trailing whitespace from the remaining text\n    trimmed_text = text_outside_quotes.strip()\n\n    # Return None if the result is an empty string, otherwise return the trimmed text\n    return trimmed_text if trimmed_text else None\n\ndef format_zp_numbers(zp_string):\n    if pd.isna(zp_string) or not isinstance(zp_string, str) or not zp_string.strip():\n        return None\n\n    # Try to extract content within double quotes first, for existing formats\n    match_quoted = re.search(r\'"([^"]*)"\', zp_string)\n    if match_quoted:\n        # If quotes found, process only the content within them\n        content_to_process =

In [149]:
"""
# Apply the function to create the new column 'text_zo'
zpp001_dummy['text_zo'] = zpp001_dummy['Purpose Code'].astype(str).apply(extract_text_outside_quotes)

# Apply the updated function to the 'Номер заявки на платеж' column to create/update 'zp no clear'
zpp001_dummy['zp no clear'] = zpp001_dummy['Purpose Code'].astype(str).apply(format_zp_numbers)
"""

"\n# Apply the function to create the new column 'text_zo'\nzpp001_dummy['text_zo'] = zpp001_dummy['Purpose Code'].astype(str).apply(extract_text_outside_quotes)\n\n# Apply the updated function to the 'Номер заявки на платеж' column to create/update 'zp no clear'\nzpp001_dummy['zp no clear'] = zpp001_dummy['Purpose Code'].astype(str).apply(format_zp_numbers)\n"

In [150]:
#zpp001_dummy = zpp001_dummy.drop(columns=['zp no clear', 'text_zo'])

In [151]:
#zpp001_dummy['Номер заявки на платеж'] = zpp001_dummy['Номер заявки на платеж'].apply(temizle_ve_cevir)

In [152]:
#zpp001_dummy.loc[zpp001_dummy['Номер заявки на платеж'] == "100000024260", 'Purpose Code'] = "31-12/31235v31-522/313132 asdvv jfjfj фывв с"

In [153]:
#zpp001_dummy[zpp001_dummy['Номер заявки на платеж']=="100000024260"]

In [154]:
zpp001_dummy.columns

Index(['﻿Валюта оплаты', 'Номер заявки на платеж', 'Тип транзакции',
       'Тип документа', 'Тип платежа', 'Статья ДДС', 'Наименование',
       'Purpose Code', 'Протокол', 'Номер договора', 'Инвойс', 'Кредитор',
       'Название кредитора', 'Оплачиваемая сумма', 'Валюта', 'Автор изменения',
       'Дата изменения', 'Создал', 'Дата создания', 'Unnamed: 19',
       'Статус согласующего', 'Оплаченная Сумма', 'Unnamed: 22', 'Дебитор',
       'Название заказчика', 'Unnamed: 25', 'Документ закупки', 'Unnamed: 27',
       'Unnamed: 28', 'Статус', 'Время', 'Время изменения', 'Дата платежа'],
      dtype='object')

In [155]:
"""
columns_to_drop = [
    'Оплачиваемая сумма_zpp001', 'Оплаченная Сумма_zpp001',
    '\ufeffВалюта оплаты_zpp001', 'Тип транзакции_zpp001',
    'Тип документа_zpp001', 'Тип платежа_zpp001', 'Статья ДДС_zpp001',
    'Наименование_zpp001', 'Purpose Code_zpp001', 'Протокол_zpp001',
    'Номер договора_zpp001', 'Инвойс_zpp001', 'Кредитор_zpp001',
    'Название кредитора_zpp001', 'Валюта_zpp001', 'Автор изменения_zpp001',
    'Создал_zpp001', 'Статус согласующего_zpp001', 'Дебитор_zpp001',
    'Название заказчика_zpp001', 'Статус_zpp001', 'text_zo_zpp001',
    'zp no clear_zpp001', 'Дата изменения_zpp001', 'Дата создания_zpp001',
    'Дата платежа_zpp001', 'Время_zpp001', 'Время изменения_zpp001',
    'Документ_закупки_cleaned', 'Номер заявки на платеж_zpp001'
]

df_B3_status.drop(columns=columns_to_drop, inplace=True, errors='ignore')

print("Specified columns dropped from df_B3_status.")
print(f"df_B3_status now has {df_B3_status.shape[1]} columns and {df_B3_status.shape[0]} rows.")
"""

'\ncolumns_to_drop = [\n    \'Оплачиваемая сумма_zpp001\', \'Оплаченная Сумма_zpp001\',\n    \'\ufeffВалюта оплаты_zpp001\', \'Тип транзакции_zpp001\',\n    \'Тип документа_zpp001\', \'Тип платежа_zpp001\', \'Статья ДДС_zpp001\',\n    \'Наименование_zpp001\', \'Purpose Code_zpp001\', \'Протокол_zpp001\',\n    \'Номер договора_zpp001\', \'Инвойс_zpp001\', \'Кредитор_zpp001\',\n    \'Название кредитора_zpp001\', \'Валюта_zpp001\', \'Автор изменения_zpp001\',\n    \'Создал_zpp001\', \'Статус согласующего_zpp001\', \'Дебитор_zpp001\',\n    \'Название заказчика_zpp001\', \'Статус_zpp001\', \'text_zo_zpp001\',\n    \'zp no clear_zpp001\', \'Дата изменения_zpp001\', \'Дата создания_zpp001\',\n    \'Дата платежа_zpp001\', \'Время_zpp001\', \'Время изменения_zpp001\',\n    \'Документ_закупки_cleaned\', \'Номер заявки на платеж_zpp001\'\n]\n\ndf_B3_status.drop(columns=columns_to_drop, inplace=True, errors=\'ignore\')\n\nprint("Specified columns dropped from df_B3_status.")\nprint(f"df_B3_status 

In [156]:
"""
zpp001_dummy.drop(columns='Документ закупки_cleaned', inplace=True, errors='ignore')
zpp001_dummy.drop(columns='zp no clear', inplace=True, errors='ignore')
"""

"\nzpp001_dummy.drop(columns='Документ закупки_cleaned', inplace=True, errors='ignore')\nzpp001_dummy.drop(columns='zp no clear', inplace=True, errors='ignore')\n"

In [157]:
#df_B3_status["zp no clear_zpp001"]

In [158]:
zpp001_dummy.columns

Index(['﻿Валюта оплаты', 'Номер заявки на платеж', 'Тип транзакции',
       'Тип документа', 'Тип платежа', 'Статья ДДС', 'Наименование',
       'Purpose Code', 'Протокол', 'Номер договора', 'Инвойс', 'Кредитор',
       'Название кредитора', 'Оплачиваемая сумма', 'Валюта', 'Автор изменения',
       'Дата изменения', 'Создал', 'Дата создания', 'Unnamed: 19',
       'Статус согласующего', 'Оплаченная Сумма', 'Unnamed: 22', 'Дебитор',
       'Название заказчика', 'Unnamed: 25', 'Документ закупки', 'Unnamed: 27',
       'Unnamed: 28', 'Статус', 'Время', 'Время изменения', 'Дата платежа'],
      dtype='object')

In [159]:
import re
import pandas as pd
import numpy as np

# --- Step 1: Define helper functions for ZP number extraction (ensure they are available) ---
def extract_text_outside_quotes(zp_string):
    if pd.isna(zp_string) or not isinstance(zp_string, str):
        return None
    text_outside_quotes = re.sub(r'"(.*?)"', '', zp_string)
    trimmed_text = text_outside_quotes.strip()
    return trimmed_text if trimmed_text else None

def format_zp_numbers(zp_string):
    if pd.isna(zp_string) or not isinstance(zp_string, str) or not zp_string.strip():
        return None
    match_quoted = re.search(r'"([^"]*)"', zp_string)
    content_to_process = match_quoted.group(1) if match_quoted else zp_string.strip()
    zp_number_patterns = re.findall(r'31-?\d+', content_to_process)
    formatted_numbers = []
    for num_str in zp_number_patterns:
        clean_num = num_str.replace('-', '')
        if clean_num.startswith('31') and len(clean_num) > 2:
            numeric_part = clean_num[2:]
            if numeric_part.isdigit():
                padded_part = numeric_part.zfill(8)
                formatted_numbers.append('31' + padded_part)
    return ';'.join(formatted_numbers) if formatted_numbers else None

# --- Step 2: Apply functions to zpp001_dummy to create 'text_zo' and 'zp no clear' ---
zpp001_dummy['text_zo'] = zpp001_dummy['Purpose Code'].astype(str).apply(extract_text_outside_quotes)
zpp001_dummy['zp no clear'] = zpp001_dummy['Purpose Code'].astype(str).apply(format_zp_numbers)

# --- Step 3: Define desired columns for aggregation from zpp001_dummy ---
numeric_cols_zpp001_to_merge = [
    'Оплачиваемая сумма',
    'Оплаченная Сумма'
]
string_cols_zpp001_to_merge = [
    '﻿Валюта оплаты', 'Номер заявки на платеж', 'Наименование', 'Purpose Code',
    'Протокол', 'Номер договора', 'Инвойс', 'Название кредитора', 'Валюта',
    'Автор изменения', 'Создал', 'Дата изменения', 'Дата создания',
    'Статус согласующего', 'Дебитор', 'Название заказчика', 'Статус',
    'text_zo', 'Дата платежа'
]

# Combine all columns to be extracted from zpp001_dummy into zpp001_temp
all_cols_for_temp = ['zp no clear'] + numeric_cols_zpp001_to_merge + string_cols_zpp001_to_merge
zpp001_temp = zpp001_dummy[all_cols_for_temp].copy()

# --- Step 4: Clean numeric columns in zpp001_temp and handle 'zp no clear' for explode ---
for col_name in numeric_cols_zpp001_to_merge:
    zpp001_temp[col_name] = (
        zpp001_temp[col_name]
        .astype(str)
        .str.replace(" ", "", regex=False)
        .str.replace(".", "", regex=False)
        .str.replace(",", ".", regex=False)
        .astype(float)
    )

# Ensure 'zp no clear' is a list of strings for explode, handling NA/empty values
zpp001_temp['zp no clear'] = zpp001_temp['zp no clear'].astype(str).apply(
    lambda x: x.split(';') if pd.notna(x) and x.strip() != '' and x.strip().lower() != 'nan' else []
)

# --- Step 5: Explode and filter zpp001_temp ---
zpp001_exploded = zpp001_temp.explode('zp no clear')
zpp001_exploded = zpp001_exploded[zpp001_exploded['zp no clear'].astype(str).str.strip() != ''].copy()

# --- Step 6: Define aggregation dictionary and create zpp001_lookup_df ---
agg_dict_for_lookup = {}
for col in numeric_cols_zpp001_to_merge:
    agg_dict_for_lookup[col] = 'sum'
for col in string_cols_zpp001_to_merge:
    agg_dict_for_lookup[col] = lambda x: '; '.join(x.dropna().astype(str).unique())

zpp001_lookup_df = zpp001_exploded.groupby('zp no clear', as_index=False).agg(agg_dict_for_lookup)

# --- Step 7: Redefine clean_document_numbers (ensure it's available) ---
def clean_document_numbers(doc_num):
    if pd.isna(doc_num):
        return pd.NA
    doc_str = str(doc_num).strip()
    if not doc_str or doc_str.lower() == 'nan':
        return pd.NA
    doc_str_cleaned = doc_str.replace('-', '')
    potential_numbers = re.findall(r'31\d+', doc_str_cleaned)
    cleaned_numbers = []
    for num_str in potential_numbers:
        if num_str.startswith('31') and len(num_str) > 2:
            numeric_part = num_str[2:]
            if numeric_part.isdigit():
                padded_part = numeric_part.zfill(8)
                if len(padded_part) > 8:
                    padded_part = padded_part[-8:]
                cleaned_numbers.append('31' + padded_part)
    return ';'.join(cleaned_numbers) if cleaned_numbers else pd.NA

# --- Step 8: Apply clean_document_numbers to df_B3_status ---
df_B3_status['Документ_закупки_cleaned'] = df_B3_status['Документ_закупки'].apply(clean_document_numbers)

# --- Step 9: Perform the merge loop and clean up ---

# Initialize new columns in df_B3_status with NaN/NA
for col in string_cols_zpp001_to_merge:
    df_B3_status[col + '_zpp001'] = pd.NA
for col in numeric_cols_zpp001_to_merge:
    df_B3_status[col + '_zpp001'] = np.nan

# Iterate through df_B3_status to merge data
for index, row in df_B3_status.iterrows():
    cleaned_doc_nums_str = row['Документ_закупки_cleaned']
    if pd.notna(cleaned_doc_nums_str):
        doc_nums = str(cleaned_doc_nums_str).split(';')

        # Filter lookup DataFrame for matching document numbers
        matching_rows = zpp001_lookup_df[zpp001_lookup_df['zp no clear'].isin(doc_nums)]

        if not matching_rows.empty:
            # Aggregate string columns by unique concatenation
            for col in string_cols_zpp001_to_merge:
                all_values = set()
                for item in matching_rows[col].dropna():
                    for sub_item in str(item).split(';'):
                        if sub_item.strip():
                            all_values.add(sub_item.strip())
                if all_values:
                    df_B3_status.loc[index, col + '_zpp001'] = ';'.join(sorted(list(all_values)))
                else:
                    df_B3_status.loc[index, col + '_zpp001'] = pd.NA

            # Aggregate numeric columns by summing
            for col in numeric_cols_zpp001_to_merge:
                sum_val = matching_rows[col].sum()
                if pd.notna(sum_val): # Assign if sum is a number (including 0.0)
                    df_B3_status.loc[index, col + '_zpp001'] = sum_val
                else: # Only assign NaN if the sum itself was NaN (e.g., all values were NaN)
                    df_B3_status.loc[index, col + '_zpp001'] = np.nan

# Drop the temporary 'Документ_закупки_cleaned' column
df_B3_status.drop(columns=['Документ_закупки_cleaned'], inplace=True)

print("zpp001 data has been successfully merged into df_B3_status.")
print(f"df_B3_status now has {df_B3_status.shape[1]} columns and {df_B3_status.shape[0]} rows.")

zpp001 data has been successfully merged into df_B3_status.
df_B3_status now has 62 columns and 387596 rows.


In [160]:
df_B3_status.columns

Index(['ГОД', 'Заявка', 'Позиция заявки', 'Материал', 'Индикатор удаления_B3',
       'ЗатребованКолич', 'Затребовал', 'Группа деблокир.', 'Дата заявки',
       'Группа закупок', 'Заказанное к-во', 'Единица измерения',
       'RU Наименование', 'TR Наименование', 'Дополнительное',
       'Инвентарный номер', 'Краткий текст', 'еще поставить (количество)',
       'Еще для поставки (стоимость)', 'Для фактурирования (колич.)',
       'Для фактурирования (стоим.)', 'ЗП_Создал', 'Группа материалов',
       'Дата_заявки', 'Материал_ME5A', 'Документ_закупки', 'Зп_Поз',
       'Индикатор удаления', 'имя_поставщика', 'Группа закупок_me2n',
       'Закуп. организация_me2n', 'Наш знак_me2n', 'Ваш код_me2n',
       'Объем заказа_me2n', 'Материал_me2n', 'Складская ЕИ_me2n',
       'ЕИ заказа на постав._me2n', 'Кол-во в ЕИ ввода', 'Дата проводки',
       'Признак 2&кол-во&ЕИ', 'Ссылка&кол-во&ЕИ', '﻿Валюта оплаты_zpp001',
       'Номер заявки на платеж_zpp001', 'Наименование_zpp001',
       'Purpose C

In [161]:
df_B3_status[['Дата платежа_zpp001','Номер заявки на платеж_zpp001']].sample(n=5)

,Дата платежа_zpp001,Номер заявки на платеж_zpp001
4016,<NA>,<NA>
299520,<NA>,<NA>
93437,<NA>,<NA>
195087,<NA>,<NA>
115668,<NA>,<NA>


In [162]:
df_B3_status["Оплачиваемая сумма_zpp001"].sample(n=5)

,Оплачиваемая сумма_zpp001
76785,NaN
326660,15160464.43
149381,NaN
853,NaN
75194,NaN


In [163]:
df_B3_status.columns

Index(['ГОД', 'Заявка', 'Позиция заявки', 'Материал', 'Индикатор удаления_B3',
       'ЗатребованКолич', 'Затребовал', 'Группа деблокир.', 'Дата заявки',
       'Группа закупок', 'Заказанное к-во', 'Единица измерения',
       'RU Наименование', 'TR Наименование', 'Дополнительное',
       'Инвентарный номер', 'Краткий текст', 'еще поставить (количество)',
       'Еще для поставки (стоимость)', 'Для фактурирования (колич.)',
       'Для фактурирования (стоим.)', 'ЗП_Создал', 'Группа материалов',
       'Дата_заявки', 'Материал_ME5A', 'Документ_закупки', 'Зп_Поз',
       'Индикатор удаления', 'имя_поставщика', 'Группа закупок_me2n',
       'Закуп. организация_me2n', 'Наш знак_me2n', 'Ваш код_me2n',
       'Объем заказа_me2n', 'Материал_me2n', 'Складская ЕИ_me2n',
       'ЕИ заказа на постав._me2n', 'Кол-во в ЕИ ввода', 'Дата проводки',
       'Признак 2&кол-во&ЕИ', 'Ссылка&кол-во&ЕИ', '﻿Валюта оплаты_zpp001',
       'Номер заявки на платеж_zpp001', 'Наименование_zpp001',
       'Purpose C

In [164]:
# 1. Yeni sütun sırasını belirle
B3status_yenisira = [
    'ГОД', 'Заявка','Позиция заявки','Дата заявки','Материал','Индикатор удаления_B3','ЗатребованКолич','Единица измерения',
    'Затребовал','Группа закупок','RU Наименование','TR Наименование','Дополнительное','Краткий текст','Инвентарный номер',
    'Дата_заявки','Группа материалов','Группа деблокир.',
    'Заказанное к-во',
    'Документ_закупки','Зп_Поз','ЗП_Создал','Индикатор удаления','имя_поставщика','Наш знак_me2n','Ваш код_me2n',
    'Объем заказа_me2n','ЕИ заказа на постав._me2n','Складская ЕИ_me2n','еще поставить (количество)','Еще для поставки (стоимость)',
    'Для фактурирования (колич.)','Для фактурирования (стоим.)','Группа закупок_me2n','Закуп. организация_me2n',
    'Кол-во в ЕИ ввода','Дата проводки','Признак 2&кол-во&ЕИ','Ссылка&кол-во&ЕИ',
    'Номер заявки на платеж_zpp001','Purpose Code_zpp001','text_zo_zpp001','Статус согласующего_zpp001','Оплачиваемая сумма_zpp001','Валюта_zpp001','Дата платежа_zpp001',
    'Оплаченная Сумма_zpp001','\ufeffВалюта оплаты_zpp001','Протокол_zpp001','Номер договора_zpp001','Создал_zpp001',
    'Дата создания_zpp001','Дата изменения_zpp001','Автор изменения_zpp001','Название кредитора_zpp001',
    'Наименование_zpp001','Инвойс_zpp001','Дебитор_zpp001','Название заказчика_zpp001'

]

# 2. DataFrame'i yeni sütun sırasına göre düzenle
df_B3_status = df_B3_status[B3status_yenisira]


In [165]:
df_B3_status.to_excel("B3_status-2026-2022.xlsx", index=False)
files.download("B3_status-2026-2022.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [166]:
df_me5a_cleaned.to_excel("df_me5a_cleaned.xlsx", index=False)
files.download("df_me5a_cleaned.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [167]:
df_zmm067_2022_2025.to_excel("zmm067_2026_2022.xlsx", index=False)
files.download("zmm067_2026_2022.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [168]:
df_me2n_2022_2025_analog.to_excel("me2n_2026_2022_analog_pos.xlsx", index=False)
files.download("me2n_2026_2022_analog_pos.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [169]:
df_me5a_2022_2025.to_excel("me5a_2026_2022.xlsx", index=False)
files.download("me5a_2026_2022.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [170]:
df_me2n_2022_2025.to_excel("me2n_2026_2022.xlsx", index=False)
files.download("me2n_2026_2022.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
birlesik_df_zmb51.to_excel("zmb51_2026_2022.xlsx", index=False)
files.download("zmb51_2026_2022.xlsx")

In [ ]:
df_calisilan.to_excel("zmm059_2026_2022.xlsx", index=False, engine="xlsxwriter")
files.download("zmm059_2026_2022.xlsx")